Projeto Final Pós Graduação Big Data - Senac Rio - 2020.

Processo de importação das Bibliotecas utilizadas no durante todo o projeto.

In [1]:
#Importação das bibliotecas utilizadas para manipulação dos dados das paginas Web.

from datetime import datetime

print(f'Ativação das bibliotecas iniciada - {datetime.now()}.')

from bs4 import BeautifulSoup
import urllib.request
import shutil
import os
import pandas as pd
import pyspark
import zipfile
import sqlalchemy
import pymysql
import json
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import * 

print(f'Ativação das bibliotecas com sucesso - {datetime.now()}.')

Ativação das bibliotecas iniciada - 2020-08-20 17:30:29.005827.
Ativação das bibliotecas com sucesso - 2020-08-20 17:30:30.391884.


Indicação do diretório para download dos arquivos Inep-Enem. 

In [ ]:
#Indicação do diretório desejado para download dos arquivos. 
#No exemplo, estamos salvando no diretório: C:\Users\Ricardo\Downloads

diretorio = input('Digite o diretório desejado para salvar os arquivos: ')
os.chdir(r'{}'.format(diretorio))
os.getcwd( )

In [ ]:
#Apos indicar o diretorio desejado, como boa pratica, sugerimos a criação da pasta onde será salvos os arquivos.
#No exemplo, estamos salvando na pasta: MicrodadosEnem.'''

url = 'http://inep.gov.br/microdados'

pasta = input('Após indicação do diretório {}, informe o nome da pasta desejada para salvar os arquivos baixados da pagina {}: '.format(diretorio, url))

try:
    os.mkdir(pasta)
    print('Pasta {} criada com sucesso!'.format(pasta))
except FileExistsError as e:
    print(f'Pasta {pasta} já existe')

In [ ]:
#Indicação do diretório onde será salvo os arquivos baixados da url.

os.chdir(r'{}\{}'.format(diretorio, pasta))
print('Os arquivos serão salvos no seguinte diretorio:', format(os.getcwd()))

Processo de Web Scraping no site do Inep, buscando os microdados do Enem.

In [ ]:
#Criação da variável que receberá todos os dados correspondente a varíavel "url".

page = urllib.request.urlopen(url)

In [ ]:
#Criação da variável "soup", do pacote BeatifulSoup, que apresenta a estutura HTML da pagina que será trabalhada.

soup = BeautifulSoup(page,'html.parser')
print(soup)

In [ ]:
#No loop "for" abaixo, o realizaremos o filtro das tags desejadas.

for link in soup.findAll('a'):
    theLink = link.get('href')
    name = link.string
    
    print(theLink)
    print(name)

In [ ]:
#No processo abaixo, realizaremos o filtro das linhas, cujo o o tipo do arquivo desejado é *.zip.

nomeArquivo = []
typeFile = '.zip'
i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            print(theLink)
            print(type(theLink))
            print(name)
            print(type(name))
            nomeArquivo.append(theLink[39:-4].replace('/','_'))
            print(type(nomeArquivo))

In [ ]:
#Processo de download dos arquivos desejados e salvos na na pasta indicada.

i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            if name is None:
                title = 'SemTítulo' + typeFile
            else:
                title = nomeArquivo[i] + typeFile
                i +=1
            try:
                with open(os.getcwd() + '/' + title,'r') as fileExists:
                    print('Arquivo ' + title + ' existente no diretório '+ os.getcwd() + ' - ' + str(datetime.now()))
                    fileExists.close()
            except FileNotFoundError:
                print('Download Iniciado - {}: '.format(datetime.now()) + title)
                docFile = open(os.getcwd() + '/' + title,'wb')
                docFile.write(urllib.request.urlopen(theLink).read())
                docFile.close()
                print('Download Concluído em {}'.format(datetime.now()))
print('Não há mais arquivos para download')

In [ ]:
#Apresentação do tamanho dos arquivos baixados.

print('Apresentação Arquivos Enem baixados no díretório: ' + os.getcwd())
contador = 0
tamanhoTotal = 0
tipo = '.zip'

def formatarTamanho(tamanho):
    base = 1024
    kilo = base
    mega = base ** 2
    giga = base ** 3
    tera = base ** 4
    
    if tamanho < kilo:
        tamanho = base
        texto = 'B'
    elif tamanho < mega:
        tamanho /= kilo
        texto = 'K'
    elif tamanho < giga:
        tamanho /= mega
        texto = 'M'
    else:
        tamanho < tera
        tamanho /= giga
        texto = 'G'
    tamanho = round(tamanho, 2)
    return f'{tamanho} {texto}'

for raiz, diretorios, arquivos in os.walk(os.getcwd()):
    for arquivo in arquivos:
        if tipo in arquivo:
            contador +=1 
            caminho = os.path.join(raiz, arquivo)
            nome, ext = os.path.splitext(arquivo)
            tamanho = os.path.getsize(caminho)
            tamanhoTotal += tamanho
                                
            if tamanho > 0:
                print()
                print('Arquivo encontrado: ', arquivo)
                print('Caminho: ', caminho)
                print('Nome: ', nome)
                print('Extensão: ', ext)
                print('Tamanho: ', formatarTamanho(tamanho))

print()
print(f'{contador} arquivo(s) encontrado(s).' + 'Tamanho Total: ' + formatarTamanho(tamanhoTotal))

Processo de Intaciação do Spark e criação dos Dataframes, atraves dos arquivos CSV's do Enem. 

In [2]:
# Instanciar Spark

print(f'Intanciando o Spark - {datetime.now()}.')

sc = SparkContext()
sqlContext = SQLContext(sc)

print(f'Spark intanciado com sucesso - {datetime.now()}.')

Intanciando o Spark - 2020-08-20 17:30:49.463251.
Spark intanciado com sucesso - 2020-08-20 17:30:53.753230.


In [3]:
# No Processo abaixo, importaremos para o dataframe, apenas para conhecer os dados e verificar o separador do microdado.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
df_enem2015_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=",", inferschema='true', encoding = 'ISO-8859-1', ).load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2015.csv')
print(f'Dataframe df_enem2015_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2016 iniciada- {datetime.now()}.')
df_enem2016_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\microdados_enem_2016.csv')
print(f'Dataframe df_enem2016_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
df_enem2017_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2017.csv')
print(f'Dataframe df_enem2017_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
df_enem2018_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2018.csv')
print(f'Dataframe df_enem2018_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
df_enem2019_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2019.csv')
print(f'Dataframe df_enem2019_completo criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Dataframes criados com sucesso - {datetime.now()}.')

Criação Dataframes Iniciado - 2020-08-20 17:30:53.782846.


Criação Dataframe do Enem 2015 iniciada - 2020-08-20 17:30:53.784838.
Dataframe df_enem2015_completo criado com sucesso - 2020-08-20 17:31:50.388501.

Criação Dataframe do Enem 2016 iniciada- 2020-08-20 17:31:50.388501.
Dataframe df_enem2016_completo criado com sucesso - 2020-08-20 17:32:48.149216.

Criação Dataframe do Enem 2017 iniciada - 2020-08-20 17:32:48.150210.
Dataframe df_enem2017_completo criado com sucesso - 2020-08-20 17:33:25.256192.

Criação Dataframe do Enem 2018 iniciada - 2020-08-20 17:33:25.257192.
Dataframe df_enem2018_completo criado com sucesso - 2020-08-20 17:33:59.076736.

Criação Dataframe do Enem 2019 iniciada - 2020-08-20 17:33:59.077733.
Dataframe df_enem2019_completo criado com sucesso - 2020-08-20 17:34:29.399801.


Dataframes criados com sucesso - 2020-08-20 17:34:29.399801.


In [4]:
# Apresentação dos registros de cada dataframe

print('Quantidade de Registros importadados para cada Dataframe')
print('')

print(f'Quantdade de Registros Dataframe Enem2015 - {datetime.now()}.')
print(df_enem2015_completo.count())
print('')

print(f'Quantdade de Registros Dataframe Enem2016 - {datetime.now()}.')
print(df_enem2016_completo.count())
print('')

print(f'Quantdade de Registros Dataframe Enem2017 - {datetime.now()}.')
print(df_enem2017_completo.count())
print('')

print(f'Quantdade de Registros Dataframe Enem2018 - {datetime.now()}.')
print(df_enem2018_completo.count())
print('')

print(f'Quantdade de Registros Dataframe Enem2019 - {datetime.now()}.')
print(df_enem2019_completo.count())


Quantidade de Registros importadados para cada Dataframe

Quantdade de Registros Dataframe Enem2015 - 2020-08-20 17:34:29.431716.
7746427

Quantdade de Registros Dataframe Enem2016 - 2020-08-20 17:34:54.084093.
8627367

Quantdade de Registros Dataframe Enem2017 - 2020-08-20 17:35:18.303845.
6731341

Quantdade de Registros Dataframe Enem2018 - 2020-08-20 17:35:34.550508.
5513747

Quantdade de Registros Dataframe Enem2019 - 2020-08-20 17:35:51.802192.
5095270


In [5]:
# Apresentação de duas linhas do dataframe df_enem2015_completo

print(f'Apresentação de duas linhas do dataframe df_enem2015_completo - {datetime.now()}.')

df_enem2015_completo.show(n=2,truncate=True)

Apresentação de duas linhas do dataframe df_enem2015_completo - 2020-08-20 17:36:06.191466.
+------------+------+-----------------------+-----------------------+----------------+----------------+---------------------------+------------+---------+----------------+----------------+---------+---------+----------------------+------------------+---------------+--------+-------+----------------+-----------------------+-----------------------+----------------+----------------+---------------+---------------+---------+---------+---------------+-----------+--------------+-----------+---------+-----------------------+-----------------+---------------------+---------------------+------------------+-----------+-----------+-----------+--------+--------------+----------+------------------+------------+------------+--------------+--------------+----------+--------------+--------------+--------+---------+--------------+---------+-----------------+---------------------+------------------------+--------

In [6]:
# Apresentação de duas linhas do dataframe df_enem2015_completo

print(f'Apresentação de duas linhas do dataframe df_enem2015_completo, classificado como linhas - {datetime.now()}.')

df_enem2015_completo.head(2)

Apresentação de duas linhas do dataframe df_enem2015_completo, classificado como linhas - 2020-08-20 17:36:06.984345.


[Row(NU_INSCRICAO=150000000376, NU_ANO=2015, CO_MUNICIPIO_RESIDENCIA=4309209, NO_MUNICIPIO_RESIDENCIA='Gravataí', CO_UF_RESIDENCIA=43, SG_UF_RESIDENCIA='RS', IN_ESTUDA_CLASSE_HOSPITALAR=0, IN_TREINEIRO=0, CO_ESCOLA=None, CO_MUNICIPIO_ESC=None, NO_MUNICIPIO_ESC=None, CO_UF_ESC=None, SG_UF_ESC=None, TP_DEPENDENCIA_ADM_ESC=None, TP_LOCALIZACAO_ESC=None, TP_SIT_FUNC_ESC=None, NU_IDADE=42, TP_SEXO='M', TP_NACIONALIDADE=1, CO_MUNICIPIO_NASCIMENTO=4209003, NO_MUNICIPIO_NASCIMENTO='Joaçaba', CO_UF_NASCIMENTO=42, SG_UF_NASCIMENTO='SC', TP_ST_CONCLUSAO=1, TP_ANO_CONCLUIU=10, TP_ESCOLA=1, TP_ENSINO=None, TP_ESTADO_CIVIL=1, TP_COR_RACA=1, IN_BAIXA_VISAO=0, IN_CEGUEIRA=0, IN_SURDEZ=0, IN_DEFICIENCIA_AUDITIVA=0, IN_SURDO_CEGUEIRA=0, IN_DEFICIENCIA_FISICA=0, IN_DEFICIENCIA_MENTAL=0, IN_DEFICIT_ATENCAO=0, IN_DISLEXIA=0, IN_GESTANTE=0, IN_LACTANTE=0, IN_IDOSO=0, IN_DISCALCULIA=0, IN_AUTISMO=0, IN_VISAO_MONOCULAR=0, IN_SABATISTA=0, IN_OUTRA_DEF=0, IN_SEM_RECURSO=0, IN_NOME_SOCIAL=0, IN_BRAILLE=0, IN_AMP

In [7]:
# Apresentação da estutura do dataframe df_enem2015_completo

print(f'Apresentação da estutura do dataframe df_enem2015_completo - {datetime.now()}.')

df_enem2015_completo.printSchema()

Apresentação da estutura do dataframe df_enem2015_completo - 2020-08-20 17:36:07.168852.
root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- IN_ESTUDA_CLASSE_HOSPITALAR: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO

In [8]:
# Apresentação das colunas do dataframe df_enem2015_completo

print(f'Apresentação das colunas do dataframe df_enem2015_completo - {datetime.now()}.')

df_enem2015_completo.columns[:]

Apresentação das colunas do dataframe df_enem2015_completo - 2020-08-20 17:36:07.205757.


['NU_INSCRICAO',
 'NU_ANO',
 'CO_MUNICIPIO_RESIDENCIA',
 'NO_MUNICIPIO_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'IN_ESTUDA_CLASSE_HOSPITALAR',
 'IN_TREINEIRO',
 'CO_ESCOLA',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'NU_IDADE',
 'TP_SEXO',
 'TP_NACIONALIDADE',
 'CO_MUNICIPIO_NASCIMENTO',
 'NO_MUNICIPIO_NASCIMENTO',
 'CO_UF_NASCIMENTO',
 'SG_UF_NASCIMENTO',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_GESTANTE',
 'IN_LACTANTE',
 'IN_IDOSO',
 'IN_DISCALCULIA',
 'IN_AUTISMO',
 'IN_VISAO_MONOCULAR',
 'IN_SABATISTA',
 'IN_OUTRA_DEF',
 'IN_SEM_RECURSO',
 'IN_NOME_SOCIAL',
 'IN_BRAILLE',
 'IN_AMPLIADA_24',
 'IN_AMPLIADA_18',
 'IN_L

In [9]:
# Descrição Dataframe df_enem2015_completo.

print('Descrição DataFrame df_enem2015_completo')
print('')

print('Descrição Idade')
df_enem2015_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2015_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2015_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2015_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2015_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2015_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2015_completo

Descrição Idade
+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|           7746352|
|   mean|23.004694338702915|
| stddev| 7.944599707765686|
|    min|                10|
|    max|                98|
+-------+------------------+

2020-08-20 17:36:30.357805.

Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          5755091|
|   mean|478.9239503771539|
| stddev|71.56005864237716|
|    min|              0.0|
|    max|            875.2|
+-------+-----------------+

2020-08-20 17:36:54.675046.

Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          5755091|
|   mean|558.4878240152929|
| stddev|70.96839011437301|
|    min|              0.0|
|    max|            850.6|
+-------+-----------------+

2020-08-20 17:37:17.2772

In [10]:
# Descrição Dataframe df_enem2016_completo.

print('Descrição DataFrame df_enem2016_completo')
print('')

print('Descrição Idade')
df_enem2016_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2016_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2016_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2016_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2016_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2016_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2016_completo

Descrição Idade
+-------+-----------------+
|summary|         NU_IDADE|
+-------+-----------------+
|  count|          8627265|
|   mean| 23.0310825041308|
| stddev|7.973406085745136|
|    min|               10|
|    max|               95|
+-------+-----------------+

2020-08-20 17:38:59.133703.

Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          5999126|
|   mean|477.6773009101653|
| stddev|72.71845884040538|
|    min|              0.0|
|    max|            871.3|
+-------+-----------------+

2020-08-20 17:39:25.766280.

Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          5999126|
|   mean|534.4409175603251|
| stddev| 74.4988507782099|
|    min|              0.0|
|    max|            859.1|
+-------+-----------------+

2020-08-20 17:39:51.900888.

Desc

In [11]:
# Descrição Dataframe df_enem2017_completo.

print('Descrição DataFrame df_enem2017_completo')
print('')

print('Descrição Idade')
df_enem2017_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2017_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2017_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2017_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2017_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2017_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2017_completo

Descrição Idade
+-------+-----------------+
|summary|         NU_IDADE|
+-------+-----------------+
|  count|          6731240|
|   mean|22.95379261473369|
| stddev|7.728937918555041|
|    min|               10|
|    max|               90|
+-------+-----------------+

2020-08-20 17:41:28.966066.

Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          4437560|
|   mean|510.8619317147265|
| stddev|72.12733741911008|
|    min|              0.0|
|    max|            885.6|
+-------+-----------------+

2020-08-20 17:41:42.801919.

Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          4701428|
|   mean|518.8259502219317|
| stddev|83.41395391030184|
|    min|              0.0|
|    max|            868.3|
+-------+-----------------+

2020-08-20 17:41:57.143544.

Desc

In [12]:
# Descrição Dataframe df_enem2018_completo.

print('Descrição DataFrame df_enem2018_completo')
print('')

print('Descrição Idade')
df_enem2018_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2018_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2018_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2018_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2018_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2018_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2018_completo

Descrição Idade
+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|           5513650|
|   mean|22.372219854361447|
| stddev| 7.594597160609549|
|    min|                10|
|    max|                98|
+-------+------------------+

2020-08-20 17:42:53.340435.

Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          3905099|
|   mean|493.6832652129926|
| stddev|  74.165826188305|
|    min|              0.0|
|    max|            869.6|
+-------+-----------------+

2020-08-20 17:43:05.529274.

Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          4148264|
|   mean|568.0977346186262|
| stddev|81.41996050791391|
|    min|              0.0|
|    max|            850.4|
+-------+-----------------+

2020-08-20 17:43:17.6041

In [13]:
# Descrição Dataframe df_enem2019_completo.

print('Descrição DataFrame df_enem2019_completo')
print('')

print('Descrição Idade')
df_enem2019_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2019_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2019_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2019_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2019_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2019_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2019_completo

Descrição Idade
+-------+-----------------+
|summary|         NU_IDADE|
+-------+-----------------+
|  count|          5095270|
|   mean|22.08743069552742|
| stddev|7.454970482110064|
|    min|                1|
|    max|               94|
+-------+-----------------+

2020-08-20 17:44:06.464697.

Descrição Nota da prova de Ciências da Natureza
+-------+------------------+
|summary|        NU_NOTA_CN|
+-------+------------------+
|  count|           3710433|
|   mean|477.82408735045146|
| stddev| 76.09774469638398|
|    min|               0.0|
|    max|             860.9|
+-------+------------------+

2020-08-20 17:44:16.745594.

Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          3923145|
|   mean|507.2486632280978|
| stddev| 82.4116343330404|
|    min|              0.0|
|    max|            835.1|
+-------+-----------------+

2020-08-20 17:44:27.1911

In [14]:
# Criação e seleção das colunas para realização das analises exploratórias

print(f'Criação da variável columns e seleção das colunas para realização das analises exploratórias - {datetime.now()}.')

columns = [
'NU_INSCRICAO', 
'NU_ANO', 
'NO_MUNICIPIO_RESIDENCIA', 
'CO_UF_RESIDENCIA', 
'SG_UF_RESIDENCIA',
'NU_IDADE', 
'TP_SEXO', 
'TP_ESTADO_CIVIL',
'TP_COR_RACA', 
'TP_NACIONALIDADE',
'NO_MUNICIPIO_NASCIMENTO',
'CO_UF_NASCIMENTO', 
'SG_UF_NASCIMENTO', 
'TP_ST_CONCLUSAO',
'TP_ESCOLA', 
'TP_ENSINO', 
'IN_TREINEIRO', 
'CO_UF_ESC', 
'SG_UF_ESC',
'TP_DEPENDENCIA_ADM_ESC', 
'TP_LOCALIZACAO_ESC', 
'IN_BAIXA_VISAO', 
'IN_CEGUEIRA', 
'IN_SURDEZ',
'IN_DEFICIENCIA_AUDITIVA', 
'IN_SURDO_CEGUEIRA',
'IN_DEFICIENCIA_FISICA', 
'IN_DEFICIENCIA_MENTAL',
'IN_DEFICIT_ATENCAO', 
'IN_DISLEXIA', 
'IN_DISCALCULIA', 
'IN_AUTISMO', 
'IN_VISAO_MONOCULAR',
'IN_OUTRA_DEF', 
'IN_GESTANTE', 
'IN_LACTANTE',
'IN_IDOSO', 
'IN_ESTUDA_CLASSE_HOSPITALAR', 
'IN_SEM_RECURSO', 
'IN_BRAILLE', 
'IN_AMPLIADA_24', 
'IN_AMPLIADA_18', 
'IN_LEDOR',
'IN_ACESSO', 
'IN_TRANSCRICAO', 
'IN_LIBRAS', 
'IN_LEITURA_LABIAL',
'IN_MESA_CADEIRA_RODAS', 
'IN_MESA_CADEIRA_SEPARADA',
'IN_APOIO_PERNA', 
'IN_GUIA_INTERPRETE', 
'IN_COMPUTADOR',
'IN_CADEIRA_ESPECIAL', 
'IN_CADEIRA_CANHOTO',
'IN_CADEIRA_ACOLCHOADA', 
'IN_PROVA_DEITADO', 
'IN_MOBILIARIO_OBESO',
'IN_LAMINA_OVERLAY', 
'IN_PROTETOR_AURICULAR', 
'IN_MEDIDOR_GLICOSE',
'IN_MAQUINA_BRAILE', 
'IN_SOROBAN', 
'IN_MARCA_PASSO', 
'IN_SONDA',
'IN_MEDICAMENTOS', 
'IN_SALA_INDIVIDUAL', 
'IN_SALA_ESPECIAL',
'IN_SALA_ACOMPANHANTE', 
'IN_MOBILIARIO_ESPECIFICO',
'IN_NOME_SOCIAL',
'NO_MUNICIPIO_PROVA', 
'CO_UF_PROVA', 
'SG_UF_PROVA',
'TP_PRESENCA_CN', 
'TP_PRESENCA_CH', 
'TP_PRESENCA_LC',
'TP_PRESENCA_MT', 
'NU_NOTA_CN', 
'NU_NOTA_CH', 
'NU_NOTA_LC',
'NU_NOTA_MT', 
'TP_LINGUA',
'TP_STATUS_REDACAO', 
'NU_NOTA_COMP1',
'NU_NOTA_COMP2', 
'NU_NOTA_COMP3', 
'NU_NOTA_COMP4', 
'NU_NOTA_COMP5',
'NU_NOTA_REDACAO', 
'Q001', 
'Q002', 
'Q005', 
'Q006',
'Q022',
'Q024', 
'Q025'
]


Criação da variável columns e seleção das colunas para realização das analises exploratórias - 2020-08-20 17:44:57.979346.


In [15]:
# No Processo abaixo, criaremos dataframes, apenas com colunas selecionadas.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
df_enem2015 = df_enem2015_completo.select(columns)
print(f'Dataframe df_enem2015 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2016 iniciada - {datetime.now()}.')
df_enem2016 = df_enem2016_completo.select(columns)
print(f'Dataframe df_enem2016 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
df_enem2017 = df_enem2017_completo.select(columns)
print(f'Dataframe df_enem2017 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
df_enem2018 = df_enem2018_completo.select(columns)
print(f'Dataframe df_enem2018 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
df_enem2019 = df_enem2019_completo.select(columns)
print(f'Dataframe df_enem2019 criado com sucesso - {datetime.now()}.')
print('')


Criação Dataframes Iniciado - 2020-08-20 17:44:58.011261.


Criação Dataframe do Enem 2015 iniciada - 2020-08-20 17:44:58.012258.
Dataframe df_enem2015 criado com sucesso - 2020-08-20 17:44:58.237854.


Criação Dataframe do Enem 2016 iniciada - 2020-08-20 17:44:58.237854.
Dataframe df_enem2016 criado com sucesso - 2020-08-20 17:44:58.483682.


Criação Dataframe do Enem 2017 iniciada - 2020-08-20 17:44:58.484680.
Dataframe df_enem2017 criado com sucesso - 2020-08-20 17:44:58.661642.


Criação Dataframe do Enem 2018 iniciada - 2020-08-20 17:44:58.661642.
Dataframe df_enem2018 criado com sucesso - 2020-08-20 17:44:58.828711.


Criação Dataframe do Enem 2019 iniciada - 2020-08-20 17:44:58.828711.
Dataframe df_enem2019 criado com sucesso - 2020-08-20 17:44:58.969537.



In [16]:
# Criação de um dataframe uníco, com a união de todos os dataframes criados. 

print(f'Criação Dataframe do df_enem iniciada - {datetime.now()}.')

df_enem = df_enem2015.unionAll(df_enem2016).unionAll(df_enem2017).unionAll(df_enem2018).unionAll(df_enem2019)

print(f'Dataframe df_enem criado com sucesso - {datetime.now()}.')

Criação Dataframe do df_enem iniciada - 2020-08-20 17:44:58.987491.
Dataframe df_enem criado com sucesso - 2020-08-20 17:44:59.060296.


In [17]:
# Total de linhas do dataframe df_enem

print(f'Quantdade de Registros Dataframe df_enem - {datetime.now()}.')

df_enem.count()

Quantdade de Registros Dataframe df_enem - 2020-08-20 17:44:59.082239.


33714152

In [18]:
# Apresentação das colunas do dataframe df_enem.

print(f'Apresentação das colunas do dataframe df_enem - {datetime.now()}.')

df_enem.columns[:]

Apresentação das colunas do dataframe df_enem - 2020-08-20 17:46:34.312555.


['NU_INSCRICAO',
 'NU_ANO',
 'NO_MUNICIPIO_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'NO_MUNICIPIO_NASCIMENTO',
 'CO_UF_NASCIMENTO',
 'SG_UF_NASCIMENTO',
 'TP_ST_CONCLUSAO',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_AUTISMO',
 'IN_VISAO_MONOCULAR',
 'IN_OUTRA_DEF',
 'IN_GESTANTE',
 'IN_LACTANTE',
 'IN_IDOSO',
 'IN_ESTUDA_CLASSE_HOSPITALAR',
 'IN_SEM_RECURSO',
 'IN_BRAILLE',
 'IN_AMPLIADA_24',
 'IN_AMPLIADA_18',
 'IN_LEDOR',
 'IN_ACESSO',
 'IN_TRANSCRICAO',
 'IN_LIBRAS',
 'IN_LEITURA_LABIAL',
 'IN_MESA_CADEIRA_RODAS',
 'IN_MESA_CADEIRA_SEPARADA',
 'IN_APOIO_PERNA',
 'IN_GUIA_INTERPRETE',
 'IN_COMPUTADOR

In [19]:
# Apresentação da estutura do dataframe df_enem

print(f'Apresentação da estutura do dataframe df_enem - {datetime.now()}.')

df_enem.printSchema()

Apresentação da estutura do dataframe df_enem - 2020-08-20 17:46:34.347973.
root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true

In [20]:
# Descrição Dataframe df_enem.

print(f'Descrição DataFrame df_enem (Dataframe com os anos 2015, 2016, 2017, 2018 e 2019) - {datetime.now()}.')
print('')

print('Descrição Idade')
df_enem.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem (Dataframe com os anos 2015, 2016, 2017, 2018 e 2019) - 2020-08-20 17:46:34.379885.

Descrição Idade
+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|          33713777|
|   mean|22.759218345663257|
| stddev| 7.788652341876025|
|    min|                 1|
|    max|                98|
+-------+------------------+

2020-08-20 17:48:10.799885.

Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|         23807309|
|   mean|486.8124319678461|
| stddev|74.22313690738514|
|    min|              0.0|
|    max|            885.6|
+-------+-----------------+

2020-08-20 17:49:45.367317.

Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|         24527054|
|   mean|538.4331498189712|
| stddev|80.97454950674937|
|    min|              0.0|
|    max| 

In [21]:
# Criar tabela temporaria e usar sql para consulta

print(f'Apresentação do processo de criação de uma tabela temporária para utilização do SQL - {datetime.now()}.')

df_enem.createOrReplaceTempView('enemtb')
teste = sqlContext.sql('SELECT NU_INSCRICAO FROM enemtb').show()

print(f'{datetime.now()}.')

Apresentação do processo de criação de uma tabela temporária para utilização do SQL - 2020-08-20 17:56:03.605018.
+------------+
|NU_INSCRICAO|
+------------+
|150000000376|
|150000003053|
|150000001161|
|150000001335|
|150000001219|
|150000002884|
|150000002510|
|150000000658|
|150000000434|
|150000000216|
|150000000274|
|150000000217|
|150000000164|
|150000000165|
|150000000218|
|150000000166|
|150000000219|
|150000000275|
|150000000220|
|150000000167|
+------------+
only showing top 20 rows

2020-08-20 17:56:04.460729.


In [22]:
# Criação das colunas de classificação do dataframe df_enem_analise. 

print(f'Criação da coluna SEXO - {datetime.now()}.')

df_enem_sexo = df_enem.withColumn('SEXO', expr("case when TP_SEXO = 'M' then 'Masculino' " + "when TP_SEXO = 'F' then 'Feminino' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SEXO - 2020-08-20 17:56:04.489654.
Criação da coluna com sucesso - 2020-08-20 17:56:04.568442.


In [23]:
print(f'Criação da coluna ESTADO_CIVIL - {datetime.now()}.')

df_enem_estado_civil = df_enem_sexo.withColumn('ESTADO_CIVIL', expr("case when TP_ESTADO_CIVIL = 0 then 'Solteiro' " + "when TP_ESTADO_CIVIL = 1 then 'Casado(a)/Mora com companheiro(a)' " + "when TP_ESTADO_CIVIL = 2 then 'Divorciado(a)/Desquitado(a)/Separado(a)' " + "when TP_ESTADO_CIVIL = 3 then 'Viúvo(a)' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna ESTADO_CIVIL - 2020-08-20 17:56:04.601354.
Criação da coluna com sucesso - 2020-08-20 17:56:04.640250.


In [24]:
print(f'Criação da coluna RACA - {datetime.now()}.')

df_enem_raca = df_enem_estado_civil.withColumn('RACA', expr("case when TP_NACIONALIDADE = 0 then 'Não declarado' " + "when TP_NACIONALIDADE = 1 then 'Branca' " + "when TP_NACIONALIDADE = 2 then 'Preta' " + "when TP_NACIONALIDADE = 3 then 'Parda' " + "when TP_NACIONALIDADE = 4 then 'Amarela' " + "when TP_NACIONALIDADE = 5 then 'Indígena ' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna RACA - 2020-08-20 17:56:04.662191.
Criação da coluna com sucesso - 2020-08-20 17:56:04.702085.


In [25]:
print(f'Criação da coluna NACIONALIDADE - {datetime.now()}.')

df_enem_nacionalidade = df_enem_raca.withColumn('NACIONALIDADE', expr("case when TP_NACIONALIDADE = 0 then 'Não informado' " + "when TP_NACIONALIDADE = 1 then 'Brasileiro(a)' " + "when TP_NACIONALIDADE = 2 then 'Brasileiro(a) Naturalizado(a)' " + "when TP_NACIONALIDADE = 3 then 'Estrangeiro(a)' " + "when TP_NACIONALIDADE = 4 then 'Brasileiro(a) Nato(a), nascido(a) no exterior' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna NACIONALIDADE - 2020-08-20 17:56:04.737988.
Criação da coluna com sucesso - 2020-08-20 17:56:04.778879.


In [26]:
print(f'Criação da coluna SITUACAO_CONCLUSAO_ENSINO_MEDIO - {datetime.now()}.')

df_enem_st_conclusao = df_enem_nacionalidade.withColumn('SITUACAO_CONCLUSAO_ENSINO_MEDIO', expr("case when TP_ST_CONCLUSAO = 1 then 'Já concluí o Ensino Médio' " + "when TP_ST_CONCLUSAO = 2 then 'Estou cursando e concluirei o Ensino Médio no ano' " + "when TP_ST_CONCLUSAO = 3 then 'Estou cursando e concluirei o Ensino Médio após este ano' " + "when TP_ST_CONCLUSAO = 4 then 'Não concluí e não estou cursando o Ensino Médio' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SITUACAO_CONCLUSAO_ENSINO_MEDIO - 2020-08-20 17:56:04.803813.
Criação da coluna com sucesso - 2020-08-20 17:56:04.845700.


In [27]:
print(f'Criação da coluna TIPO_ESCOLA - {datetime.now()}.')

df_enem_tp_escola = df_enem_st_conclusao.withColumn('TIPO_ESCOLA', expr("case when TP_ESCOLA = 1 then 'Não Respondeu' " + "when TP_ESCOLA = 2 then 'Pública' " + "when TP_ESCOLA = 3 then 'Privada' " + "when TP_ESCOLA = 4 then 'Exterior' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna TIPO_ESCOLA - 2020-08-20 17:56:04.881604.
Criação da coluna com sucesso - 2020-08-20 17:56:04.925488.


In [28]:
print(f'Criação da coluna TIPO_ENSINO - {datetime.now()}.')

df_enem_tp_ensino = df_enem_tp_escola.withColumn('TIPO_ENSINO', expr("case when TP_ENSINO = 1 then 'Ensino Regular' " + "when TP_ENSINO = 2 then 'Educação Especial - Modalidade Substitutiva' " + "when TP_ENSINO = 3 then 'Educação de Jovens e Adultos' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna TIPO_ENSINO - 2020-08-20 17:56:04.965380.
Criação da coluna com sucesso - 2020-08-20 17:56:05.003279.


In [29]:
print(f'Criação da coluna TREINEIRO - {datetime.now()}.')

df_enem_in_treineiro = df_enem_tp_ensino.withColumn('TREINEIRO', expr("case when IN_TREINEIRO = 0 then 'Não' " + "when IN_TREINEIRO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna TREINEIRO - 2020-08-20 17:56:05.038187.
Criação da coluna com sucesso - 2020-08-20 17:56:05.080074.


In [30]:
print(f'Criação da coluna DEPENDENCIA_ADMINISTRATIVA_ESCOLA - {datetime.now()}.')

df_enem_tp_dependencia_adm_esc = df_enem_in_treineiro.withColumn('DEPENDENCIA_ADMINISTRATIVA_ESCOLA', expr("case when TP_DEPENDENCIA_ADM_ESC = 1 then 'Federal' " + "when TP_DEPENDENCIA_ADM_ESC = 2 then 'Estadual' " + "when TP_DEPENDENCIA_ADM_ESC = 3 then 'Municipal' " + "when TP_DEPENDENCIA_ADM_ESC = 4 then 'Privada' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DEPENDENCIA_ADMINISTRATIVA_ESCOLA - 2020-08-20 17:56:05.115978.
Criação da coluna com sucesso - 2020-08-20 17:56:05.154873.


In [31]:
print(f'Criação da coluna LOCALIZACAO_ESCOLA - {datetime.now()}.')

df_enem_tp_localizacao_esc = df_enem_tp_dependencia_adm_esc.withColumn('LOCALIZACAO_ESCOLA', expr("case when TP_LOCALIZACAO_ESC = 1 then 'Urbana' " + "when TP_LOCALIZACAO_ESC = 2 then 'Rural' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LOCALIZACAO_ESCOLA - 2020-08-20 17:56:05.176814.
Criação da coluna com sucesso - 2020-08-20 17:56:05.218703.


In [32]:
print(f'Criação da coluna BAIXA_VISAO - {datetime.now()}.')

df_enem_in_baixa_visao = df_enem_tp_localizacao_esc.withColumn('BAIXA_VISAO', expr("case when IN_BAIXA_VISAO = 0 then 'Não' " + "when IN_BAIXA_VISAO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna BAIXA_VISAO - 2020-08-20 17:56:05.250618.
Criação da coluna com sucesso - 2020-08-20 17:56:05.290511.


In [33]:
print(f'Criação da coluna CEGUEIRA - {datetime.now()}.')

df_enem_in_cegueira = df_enem_in_baixa_visao.withColumn('CEGUEIRA', expr("case when IN_CEGUEIRA = 0 then 'Não' " + "when IN_CEGUEIRA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')


Criação da coluna CEGUEIRA - 2020-08-20 17:56:05.316441.
Criação da coluna com sucesso - 2020-08-20 17:56:05.349355.


In [34]:
print(f'Criação da coluna SURDEZ - {datetime.now()}.')

df_enem_in_surdez = df_enem_in_cegueira.withColumn('SURDEZ', expr("case when IN_SURDEZ = 0 then 'Não' " + "when IN_SURDEZ = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SURDEZ - 2020-08-20 17:56:05.376281.
Criação da coluna com sucesso - 2020-08-20 17:56:05.415177.


In [35]:
print(f'Criação da coluna DEFICIENCIA_AUDITIVA - {datetime.now()}.')

df_enem_in_deficiencia_auditiva = df_enem_in_surdez.withColumn('DEFICIENCIA_AUDITIVA', expr("case when IN_DEFICIENCIA_AUDITIVA = 0 then 'Não' " + "when IN_DEFICIENCIA_AUDITIVA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DEFICIENCIA_AUDITIVA - 2020-08-20 17:56:05.440110.
Criação da coluna com sucesso - 2020-08-20 17:56:05.478010.


In [36]:
print(f'Criação da coluna SURDO_CEGUEIRA - {datetime.now()}.')

df_enem_in_surdo_cegueira = df_enem_in_deficiencia_auditiva.withColumn('SURDO_CEGUEIRA', expr("case when IN_SURDO_CEGUEIRA = 0 then 'Não' " + "when IN_SURDO_CEGUEIRA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SURDO_CEGUEIRA - 2020-08-20 17:56:05.501947.
Criação da coluna com sucesso - 2020-08-20 17:56:05.540840.


In [37]:
print(f'Criação da coluna DEFICIENCIA_FISICA - {datetime.now()}.')

df_enem_in_deficiencia_fisica = df_enem_in_surdo_cegueira.withColumn('DEFICIENCIA_FISICA', expr("case when IN_DEFICIENCIA_FISICA = 0 then 'Não' " + "when IN_DEFICIENCIA_FISICA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DEFICIENCIA_FISICA - 2020-08-20 17:56:05.563779.
Criação da coluna com sucesso - 2020-08-20 17:56:05.603673.


In [38]:
print(f'Criação da coluna DEFICIENCIA_MENTAL - {datetime.now()}.')

df_enem_in_deficiencia_mental = df_enem_in_deficiencia_fisica.withColumn('DEFICIENCIA_MENTAL', expr("case when IN_DEFICIENCIA_MENTAL = 0 then 'Não' " + "when IN_DEFICIENCIA_MENTAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DEFICIENCIA_MENTAL - 2020-08-20 17:56:05.624617.
Criação da coluna com sucesso - 2020-08-20 17:56:05.664510.


In [39]:
print(f'Criação da coluna DEFICIT_ATENCAO - {datetime.now()}.')

df_enem_in_deficit_atencao = df_enem_in_deficiencia_mental.withColumn('DEFICIT_ATENCAO', expr("case when IN_DEFICIT_ATENCAO = 0 then 'Não' " + "when IN_DEFICIT_ATENCAO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DEFICIT_ATENCAO - 2020-08-20 17:56:05.687449.
Criação da coluna com sucesso - 2020-08-20 17:56:05.723352.


In [40]:
print(f'Criação da coluna DISLEXIA - {datetime.now()}.')

df_enem_in_dislexia = df_enem_in_deficit_atencao.withColumn('DISLEXIA', expr("case when IN_DISLEXIA = 0 then 'Não' " + "when IN_DISLEXIA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DISLEXIA - 2020-08-20 17:56:05.746292.
Criação da coluna com sucesso - 2020-08-20 17:56:05.786186.


In [41]:
print(f'Criação da coluna DISCALCULIA - {datetime.now()}.')

df_enem_in_discalculia = df_enem_in_dislexia.withColumn('DISCALCULIA', expr("case when IN_DISCALCULIA = 0 then 'Não' " + "when IN_DISCALCULIA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna DISCALCULIA - 2020-08-20 17:56:05.810122.
Criação da coluna com sucesso - 2020-08-20 17:56:05.845028.


In [42]:
print(f'Criação da coluna AUTISMO - {datetime.now()}.')

df_enem_in_autismo = df_enem_in_discalculia.withColumn('AUTISMO', expr("case when IN_AUTISMO = 0 then 'Não' " + "when IN_AUTISMO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna AUTISMO - 2020-08-20 17:56:05.874946.
Criação da coluna com sucesso - 2020-08-20 17:56:05.915837.


In [43]:
print(f'Criação da coluna VISAO_MONOCULAR - {datetime.now()}.')

df_enem_in_visao_monocular = df_enem_in_autismo.withColumn('VISAO_MONOCULAR', expr("case when IN_VISAO_MONOCULAR = 0 then 'Não' " + "when IN_VISAO_MONOCULAR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna VISAO_MONOCULAR - 2020-08-20 17:56:05.934787.
Criação da coluna com sucesso - 2020-08-20 17:56:05.972686.


In [44]:
print(f'Criação da coluna OUTRA_DEF - {datetime.now()}.')

df_enem_in_outra_def = df_enem_in_visao_monocular.withColumn('OUTRA_DEF', expr("case when IN_OUTRA_DEF = 0 then 'Não' " + "when IN_OUTRA_DEF = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna OUTRA_DEF - 2020-08-20 17:56:05.998616.
Criação da coluna com sucesso - 2020-08-20 17:56:06.040506.


In [45]:
print(f'Criação da coluna GESTANTE - {datetime.now()}.')

df_enem_in_gestante = df_enem_in_outra_def.withColumn('GESTANTE', expr("case when IN_GESTANTE = 0 then 'Não' " + "when IN_GESTANTE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna GESTANTE - 2020-08-20 17:56:06.061448.
Criação da coluna com sucesso - 2020-08-20 17:56:06.113311.


In [46]:
print(f'Criação da coluna LACTANTE - {datetime.now()}.')

df_enem_in_lactante = df_enem_in_gestante.withColumn('LACTANTE', expr("case when IN_LACTANTE = 0 then 'Não' " + "when IN_LACTANTE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LACTANTE - 2020-08-20 17:56:06.140244.
Criação da coluna com sucesso - 2020-08-20 17:56:06.178136.


In [47]:
print(f'Criação da coluna IDOSO - {datetime.now()}.')

df_enem_in_idoso = df_enem_in_lactante.withColumn('IDOSO', expr("case when IN_IDOSO = 0 then 'Não' " + "when IN_IDOSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna IDOSO - 2020-08-20 17:56:06.221022.
Criação da coluna com sucesso - 2020-08-20 17:56:06.258920.


In [48]:
print(f'Criação da coluna ESTUDA_CLASSE_HOSPITALAR - {datetime.now()}.')

df_enem_in_estuda_classe_hospitalar = df_enem_in_idoso.withColumn('ESTUDA_CLASSE_HOSPITALAR', expr("case when IN_ESTUDA_CLASSE_HOSPITALAR = 0 then 'Não' " + "when IN_ESTUDA_CLASSE_HOSPITALAR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna ESTUDA_CLASSE_HOSPITALAR - 2020-08-20 17:56:06.278867.
Criação da coluna com sucesso - 2020-08-20 17:56:06.318761.


In [49]:
print(f'Criação da coluna SEM_RECURSO - {datetime.now()}.')

df_enem_in_sem_recurso = df_enem_in_estuda_classe_hospitalar.withColumn('SEM_RECURSO', expr("case when IN_SEM_RECURSO = 0 then 'Não' " + "when IN_SEM_RECURSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SEM_RECURSO - 2020-08-20 17:56:06.340702.
Criação da coluna com sucesso - 2020-08-20 17:56:06.374611.


In [50]:
print(f'Criação da coluna BRAILLE - {datetime.now()}.')

df_enem_in_braille = df_enem_in_sem_recurso.withColumn('BRAILLE', expr("case when IN_BRAILLE = 0 then 'Não' " + "when IN_BRAILLE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna BRAILLE - 2020-08-20 17:56:06.400542.
Criação da coluna com sucesso - 2020-08-20 17:56:06.442430.


In [51]:
print(f'Criação da coluna AMPLIADA_24 - {datetime.now()}.')

df_enem_in_ampliada_24 = df_enem_in_braille.withColumn('AMPLIADA_24', expr("case when IN_AMPLIADA_24 = 0 then 'Não' " + "when IN_AMPLIADA_24 = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna AMPLIADA_24 - 2020-08-20 17:56:06.469358.
Criação da coluna com sucesso - 2020-08-20 17:56:06.509253.


In [52]:
print(f'Criação da coluna AMPLIADA_18 - {datetime.now()}.')

df_enem_in_ampliada_18 = df_enem_in_ampliada_24.withColumn('AMPLIADA_18', expr("case when IN_AMPLIADA_18 = 0 then 'Não' " + "when IN_AMPLIADA_18 = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna AMPLIADA_18 - 2020-08-20 17:56:06.528200.
Criação da coluna com sucesso - 2020-08-20 17:56:06.566098.


In [53]:
print(f'Criação da coluna LEDOR - {datetime.now()}.')

df_enem_in_ledor = df_enem_in_ampliada_18.withColumn('LEDOR', expr("case when IN_LEDOR = 0 then 'Não' " + "when IN_LEDOR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LEDOR - 2020-08-20 17:56:06.593026.
Criação da coluna com sucesso - 2020-08-20 17:56:06.637906.


In [54]:
print(f'Criação da coluna ACESSO - {datetime.now()}.')

df_enem_in_acesso = df_enem_in_ledor.withColumn('ACESSO', expr("case when IN_ACESSO = 0 then 'Não' " + "when IN_ACESSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna ACESSO - 2020-08-20 17:56:06.652867.
Criação da coluna com sucesso - 2020-08-20 17:56:06.689767.


In [55]:
print(f'Criação da coluna TRANSCRICAO - {datetime.now()}.')

df_enem_in_transcricao = df_enem_in_acesso.withColumn('TRANSCRICAO', expr("case when IN_TRANSCRICAO = 0 then 'Não' " + "when IN_TRANSCRICAO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna TRANSCRICAO - 2020-08-20 17:56:06.718690.
Criação da coluna com sucesso - 2020-08-20 17:56:06.757586.


In [56]:
print(f'Criação da coluna LIBRAS - {datetime.now()}.')

df_enem_in_libras = df_enem_in_transcricao.withColumn('LIBRAS', expr("case when IN_LIBRAS = 0 then 'Não' " + "when IN_LIBRAS = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LIBRAS - 2020-08-20 17:56:06.778530.
Criação da coluna com sucesso - 2020-08-20 17:56:06.814435.


In [57]:
print(f'Criação da coluna LEITURA_LABIAL - {datetime.now()}.')

df_enem_in_leitura_labial = df_enem_in_libras.withColumn('LEITURA_LABIAL', expr("case when IN_LEITURA_LABIAL = 0 then 'Não' " + "when IN_LEITURA_LABIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LEITURA_LABIAL - 2020-08-20 17:56:06.842359.
Criação da coluna com sucesso - 2020-08-20 17:56:06.876269.


In [58]:
print(f'Criação da coluna MESA_CADEIRA_RODAS - {datetime.now()}.')

df_enem_in_mesa_cadeira_rodas = df_enem_in_leitura_labial.withColumn('MESA_CADEIRA_RODAS', expr("case when IN_MESA_CADEIRA_RODAS = 0 then 'Não' " + "when IN_MESA_CADEIRA_RODAS = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MESA_CADEIRA_RODAS - 2020-08-20 17:56:06.905193.
Criação da coluna com sucesso - 2020-08-20 17:56:06.940104.


In [59]:
print(f'Criação da coluna MESA_CADEIRA_SEPARADA - {datetime.now()}.')

df_enem_in_mesa_cadeira_separada = df_enem_in_mesa_cadeira_rodas.withColumn('MESA_CADEIRA_SEPARADA', expr("case when IN_MESA_CADEIRA_SEPARADA = 0 then 'Não' " + "when IN_MESA_CADEIRA_SEPARADA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MESA_CADEIRA_SEPARADA - 2020-08-20 17:56:06.964034.
Criação da coluna com sucesso - 2020-08-20 17:56:07.000935.


In [60]:
print(f'Criação da coluna APOIO_PERNA - {datetime.now()}.')

df_enem_in_apoio_perna = df_enem_in_mesa_cadeira_separada.withColumn('APOIO_PERNA', expr("case when IN_APOIO_PERNA = 0 then 'Não' " + "when IN_APOIO_PERNA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna APOIO_PERNA - 2020-08-20 17:56:07.027863.
Criação da coluna com sucesso - 2020-08-20 17:56:07.071745.


In [61]:
print(f'Criação da coluna GUIA_INTERPRETE - {datetime.now()}.')

df_enem_in_guia_interprete = df_enem_in_apoio_perna.withColumn('GUIA_INTERPRETE', expr("case when IN_GUIA_INTERPRETE = 0 then 'Não' " + "when IN_GUIA_INTERPRETE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna GUIA_INTERPRETE - 2020-08-20 17:56:07.091693.
Criação da coluna com sucesso - 2020-08-20 17:56:07.133582.


In [62]:
print(f'Criação da coluna COMPUTADOR - {datetime.now()}.')

df_enem_in_computador = df_enem_in_guia_interprete.withColumn('COMPUTADOR', expr("case when IN_COMPUTADOR = 0 then 'Não' " + "when IN_COMPUTADOR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna COMPUTADOR - 2020-08-20 17:56:07.155522.
Criação da coluna com sucesso - 2020-08-20 17:56:07.203394.


In [63]:
print(f'Criação da coluna CADEIRA_ESPECIAL - {datetime.now()}.')

df_enem_in_cadeira_especial = df_enem_in_computador.withColumn('CADEIRA_ESPECIAL', expr("case when IN_CADEIRA_ESPECIAL = 0 then 'Não' " + "when IN_CADEIRA_ESPECIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna CADEIRA_ESPECIAL - 2020-08-20 17:56:07.233314.
Criação da coluna com sucesso - 2020-08-20 17:56:07.276200.


In [64]:
print(f'Criação da coluna CADEIRA_CANHOTO - {datetime.now()}.')

df_enem_in_cadeira_canhoto = df_enem_in_cadeira_especial.withColumn('CADEIRA_CANHOTO', expr("case when IN_CADEIRA_CANHOTO = 0 then 'Não' " + "when IN_CADEIRA_CANHOTO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna CADEIRA_CANHOTO - 2020-08-20 17:56:07.297144.
Criação da coluna com sucesso - 2020-08-20 17:56:07.337036.


In [65]:
print(f'Criação da coluna CADEIRA_ACOLCHOADA - {datetime.now()}.')

df_enem_in_cadeira_acolchoada = df_enem_in_cadeira_canhoto.withColumn('CADEIRA_ACOLCHOADA', expr("case when IN_CADEIRA_ACOLCHOADA = 0 then 'Não' " + "when IN_CADEIRA_ACOLCHOADA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna CADEIRA_ACOLCHOADA - 2020-08-20 17:56:07.355986.
Criação da coluna com sucesso - 2020-08-20 17:56:07.401864.


In [66]:
print(f'Criação da coluna PROVA_DEITADO - {datetime.now()}.')

df_enem_in_prova_deitado = df_enem_in_cadeira_acolchoada.withColumn('PROVA_DEITADO', expr("case when IN_PROVA_DEITADO = 0 then 'Não' " + "when IN_PROVA_DEITADO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PROVA_DEITADO - 2020-08-20 17:56:07.421810.
Criação da coluna com sucesso - 2020-08-20 17:56:07.459708.


In [67]:
print(f'Criação da coluna MOBILIARIO_OBESO - {datetime.now()}.')

df_enem_in_mobiliario_obeso = df_enem_in_prova_deitado.withColumn('MOBILIARIO_OBESO', expr("case when IN_MOBILIARIO_OBESO = 0 then 'Não' " + "when IN_MOBILIARIO_OBESO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MOBILIARIO_OBESO - 2020-08-20 17:56:07.484643.
Criação da coluna com sucesso - 2020-08-20 17:56:07.524536.


In [68]:
print(f'Criação da coluna LAMINA_OVERLAY - {datetime.now()}.')

df_enem_in_lamina_overlay = df_enem_in_mobiliario_obeso.withColumn('LAMINA_OVERLAY', expr("case when IN_LAMINA_OVERLAY = 0 then 'Não' " + "when IN_LAMINA_OVERLAY = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LAMINA_OVERLAY - 2020-08-20 17:56:07.544482.
Criação da coluna com sucesso - 2020-08-20 17:56:07.584375.


In [69]:
print(f'Criação da coluna PROTETOR_AURICULAR - {datetime.now()}.')

df_enem_in_protetor_auricular = df_enem_in_lamina_overlay.withColumn('PROTETOR_AURICULAR', expr("case when IN_PROTETOR_AURICULAR = 0 then 'Não' " + "when IN_PROTETOR_AURICULAR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PROTETOR_AURICULAR - 2020-08-20 17:56:07.609309.
Criação da coluna com sucesso - 2020-08-20 17:56:07.655186.


In [70]:
print(f'Criação da coluna MEDIDOR_GLICOSE - {datetime.now()}.')

df_enem_in_medidor_glicose = df_enem_in_protetor_auricular.withColumn('MEDIDOR_GLICOSE', expr("case when IN_MEDIDOR_GLICOSE = 0 then 'Não' " + "when IN_MEDIDOR_GLICOSE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MEDIDOR_GLICOSE - 2020-08-20 17:56:07.670146.
Criação da coluna com sucesso - 2020-08-20 17:56:07.711036.


In [71]:
print(f'Criação da coluna MAQUINA_BRAILE - {datetime.now()}.')

df_enem_in_maquina_braile = df_enem_in_medidor_glicose.withColumn('MAQUINA_BRAILE', expr("case when IN_MAQUINA_BRAILE = 0 then 'Não' " + "when IN_MAQUINA_BRAILE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MAQUINA_BRAILE - 2020-08-20 17:56:07.736968.
Criação da coluna com sucesso - 2020-08-20 17:56:07.783841.


In [72]:
print(f'Criação da coluna SOROBAN - {datetime.now()}.')

df_enem_in_soroban = df_enem_in_maquina_braile.withColumn('SOROBAN', expr("case when IN_SOROBAN = 0 then 'Não' " + "when IN_SOROBAN = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SOROBAN - 2020-08-20 17:56:07.798802.
Criação da coluna com sucesso - 2020-08-20 17:56:07.839692.


In [73]:
print(f'Criação da coluna MARCA_PASSO - {datetime.now()}.')

df_enem_in_marca_passo = df_enem_in_soroban.withColumn('MARCA_PASSO', expr("case when IN_MARCA_PASSO = 0 then 'Não' " + "when IN_MARCA_PASSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MARCA_PASSO - 2020-08-20 17:56:07.858641.
Criação da coluna com sucesso - 2020-08-20 17:56:07.899533.


In [74]:
print(f'Criação da coluna SONDA - {datetime.now()}.')

df_enem_in_sonda = df_enem_in_marca_passo.withColumn('SONDA', expr("case when IN_SONDA = 0 then 'Não' " + "when IN_SONDA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SONDA - 2020-08-20 17:56:07.925463.
Criação da coluna com sucesso - 2020-08-20 17:56:07.973336.


In [75]:
print(f'Criação da coluna MEDICAMENTOS - {datetime.now()}.')

df_enem_in_medicamentos = df_enem_in_sonda.withColumn('MEDICAMENTOS', expr("case when IN_MEDICAMENTOS = 0 then 'Não' " + "when IN_MEDICAMENTOS = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MEDICAMENTOS - 2020-08-20 17:56:07.998269.
Criação da coluna com sucesso - 2020-08-20 17:56:08.035171.


In [76]:
print(f'Criação da coluna SALA_INDIVIDUAL - {datetime.now()}.')

df_enem_in_sala_individual = df_enem_in_medicamentos.withColumn('SALA_INDIVIDUAL', expr("case when IN_SALA_INDIVIDUAL = 0 then 'Não' " + "when IN_SALA_INDIVIDUAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SALA_INDIVIDUAL - 2020-08-20 17:56:08.063094.
Criação da coluna com sucesso - 2020-08-20 17:56:08.103987.


In [77]:
print(f'Criação da coluna SALA_ESPECIAL - {datetime.now()}.')

df_enem_in_sala_especial = df_enem_in_sala_individual.withColumn('SALA_ESPECIAL', expr("case when IN_SALA_ESPECIAL = 0 then 'Não' " + "when IN_SALA_ESPECIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SALA_ESPECIAL - 2020-08-20 17:56:08.129918.
Criação da coluna com sucesso - 2020-08-20 17:56:08.170808.


In [78]:
print(f'Criação da coluna SALA_ACOMPANHANTE - {datetime.now()}.')

df_enem_in_sala_acompanhante = df_enem_in_sala_especial.withColumn('SALA_ACOMPANHANTE', expr("case when IN_SALA_ACOMPANHANTE = 0 then 'Não' " + "when IN_SALA_ACOMPANHANTE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna SALA_ACOMPANHANTE - 2020-08-20 17:56:08.189756.
Criação da coluna com sucesso - 2020-08-20 17:56:08.227656.


In [79]:
print(f'Criação da coluna MOBILIARIO_ESPECIFICO - {datetime.now()}.')

df_enem_in_mobiliario_especifico = df_enem_in_sala_acompanhante.withColumn('MOBILIARIO_ESPECIFICO', expr("case when IN_MOBILIARIO_ESPECIFICO = 0 then 'Não' " + "when IN_MOBILIARIO_ESPECIFICO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MOBILIARIO_ESPECIFICO - 2020-08-20 17:56:08.249596.
Criação da coluna com sucesso - 2020-08-20 17:56:08.296472.


In [80]:
print(f'Criação da coluna NOME_SOCIAL - {datetime.now()}.')

df_enem_in_nome_social = df_enem_in_mobiliario_especifico.withColumn('NOME_SOCIAL', expr("case when IN_NOME_SOCIAL = 0 then 'Não' " + "when IN_NOME_SOCIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna NOME_SOCIAL - 2020-08-20 17:56:08.314425.
Criação da coluna com sucesso - 2020-08-20 17:56:08.352323.


In [81]:
print(f'Criação da coluna PRESENCA_CIENCIAS_NATUREZA - {datetime.now()}.')

df_enem_tp_presenca_cn = df_enem_in_nome_social.withColumn('PRESENCA_CIENCIAS_NATUREZA', expr("case when TP_PRESENCA_CN = 0 then 'Faltou à prova' " + "when TP_PRESENCA_CN = 1 then 'Presente na prova' " + "when TP_PRESENCA_CN = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PRESENCA_CIENCIAS_NATUREZA - 2020-08-20 17:56:08.377255.
Criação da coluna com sucesso - 2020-08-20 17:56:08.422135.


In [82]:
print(f'Criação da coluna PRESENCA_CIENCIAS_HUMANAS - {datetime.now()}.')

df_enem_tp_presenca_ch = df_enem_tp_presenca_cn.withColumn('PRESENCA_CIENCIAS_HUMANAS', expr("case when TP_PRESENCA_CH = 0 then 'Faltou à prova' " + "when TP_PRESENCA_CH = 1 then 'Presente na prova' " + "when TP_PRESENCA_CH = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PRESENCA_CIENCIAS_HUMANAS - 2020-08-20 17:56:08.443078.
Criação da coluna com sucesso - 2020-08-20 17:56:08.490951.


In [83]:
print(f'Criação da coluna PRESENCA_LINGUAGENS_CODIGOS - {datetime.now()}.')

df_enem_tp_presenca_lc = df_enem_tp_presenca_ch.withColumn('PRESENCA_LINGUAGENS_CODIGOS', expr("case when TP_PRESENCA_LC = 0 then 'Faltou à prova' " + "when TP_PRESENCA_LC = 1 then 'Presente na prova' " + "when TP_PRESENCA_LC = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PRESENCA_LINGUAGENS_CODIGOS - 2020-08-20 17:56:08.516881.
Criação da coluna com sucesso - 2020-08-20 17:56:08.557772.


In [84]:
print(f'Criação da coluna PRESENCA_MATEMATICA - {datetime.now()}.')

df_enem_tp_presenca_mt = df_enem_tp_presenca_lc.withColumn('PRESENCA_MATEMATICA', expr("case when TP_PRESENCA_MT = 0 then 'Faltou à prova' " + "when TP_PRESENCA_MT = 1 then 'Presente na prova' " + "when TP_PRESENCA_MT = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PRESENCA_MATEMATICA - 2020-08-20 17:56:08.576721.
Criação da coluna com sucesso - 2020-08-20 17:56:08.622599.


In [85]:
print(f'Criação da coluna LINGUA - {datetime.now()}.')

df_enem_tp_lingua = df_enem_tp_presenca_mt.withColumn('LINGUA', expr("case when TP_LINGUA = 0 then 'Inglês' " + "when TP_LINGUA = 1 then 'Espanhol' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna LINGUA - 2020-08-20 17:56:08.645546.
Criação da coluna com sucesso - 2020-08-20 17:56:08.684469.


In [86]:
print(f'Criação da coluna STATUS_REDACAO - {datetime.now()}.')

df_enem_tp_status_redacao = df_enem_tp_lingua.withColumn('STATUS_REDACAO', expr("case when TP_STATUS_REDACAO = 1 then 'Sem problemas' " + "when TP_STATUS_REDACAO = 2 then 'Anulada' " + "when TP_STATUS_REDACAO = 3 then 'Cópia Texto Motivador' " + "when TP_STATUS_REDACAO = 4 then 'Em Branco' " + "when TP_STATUS_REDACAO = 6 then 'Fuga ao tema' " + "when TP_STATUS_REDACAO = 7 then 'Não atendimento ao tipo textual' " + "when TP_STATUS_REDACAO = 8 then 'Texto insuficiente' " + "when TP_STATUS_REDACAO = 9 then 'Parte desconectada' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna STATUS_REDACAO - 2020-08-20 17:56:08.702386.
Criação da coluna com sucesso - 2020-08-20 17:56:08.746269.


In [87]:
print(f'Criação da coluna PAI_ESTUDOU - {datetime.now()}.')

df_enem_q001 = df_enem_tp_status_redacao.withColumn('PAI_ESTUDOU', expr("case when Q001 = 'A' then 'Nunca estudou.' " + "when Q001 = 'B' then 'Não completou a 4ª série/5º ano do Ensino Fundamental.' " + "when Q001 = 'C' then 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.' " + "when Q001 = 'D' then 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.' " + "when Q001 = 'E' then 'Completou o Ensino Médio, mas não completou a Faculdade.' " + "when Q001 = 'F' then 'Completou a Faculdade, mas não completou a Pós-graduação.' " + "when Q001 = 'G' then 'Completou a Pós-graduação.' " + "when Q001 = 'H' then 'Não sei.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna PAI_ESTUDOU - 2020-08-20 17:56:08.766214.
Criação da coluna com sucesso - 2020-08-20 17:56:08.814117.


In [88]:
print(f'Criação da coluna MAE_ESTUDOU - {datetime.now()}.')

df_enem_q002 = df_enem_q001.withColumn('MAE_ESTUDOU', expr("case when Q002 = 'A' then 'Nunca estudou.' " + "when Q002 = 'B' then 'Não completou a 4ª série/5º ano do Ensino Fundamental.' " + "when Q002 = 'C' then 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.' " + "when Q002 = 'D' then 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.' " + "when Q002 = 'E' then 'Completou o Ensino Médio, mas não completou a Faculdade.' " + "when Q002 = 'F' then 'Completou a Faculdade, mas não completou a Pós-graduação.' " + "when Q002 = 'G' then 'Completou a Pós-graduação.' " + "when Q002 = 'H' then 'Não sei.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna MAE_ESTUDOU - 2020-08-20 17:56:08.829046.
Criação da coluna com sucesso - 2020-08-20 17:56:08.875923.


In [89]:
print(f'Criação da coluna QTDE_MORADOR_RESIDENCIA - {datetime.now()}.')

df_enem_q005 = df_enem_q002.withColumn('QTDE_MORADOR_RESIDENCIA', expr("case when Q005 = 1 then '1, pois moro sozinho(a).' " + "when Q005 = 2 then '2' " + "when Q005 = 3 then '3' " + "when Q005 = 4 then '4' " + "when Q005 = 5 then '5' " + "when Q005 = 6 then '6' " + "when Q005 = 7 then '7' " + "when Q005 = 8 then '8' " + "when Q005 = 9 then '9' " + "when Q005 = 10 then '10' " + "when Q005 = 11 then '11' " + "when Q005 = 12 then '12' " + "when Q005 = 13 then '13' " + "when Q005 = 14 then '14' " + "when Q005 = 15 then '15' " + "when Q005 = 16 then '16' " + "when Q005 = 17 then '17' " + "when Q005 = 18 then '18' " + "when Q005 = 19 then '19' " + "when Q005 = 20 then '20' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna QTDE_MORADOR_RESIDENCIA - 2020-08-20 17:56:08.891880.
Criação da coluna com sucesso - 2020-08-20 17:56:08.949723.


In [90]:
print(f'Criação da coluna RENDA_MENSAL_FAMILIAR - {datetime.now()}.')

df_enem_q006 = df_enem_q005.withColumn('RENDA_MENSAL_FAMILIAR', expr("case when Q006 = 'A' then 'Nenhuma Renda.' " + "when Q006 = 'B' then 'Até 1,0 salário mínimo.' " + "when Q006 = 'C' then 'Acima de 1,0 salário mínimo até 1,5.' " + "when Q006 = 'D' then 'Acima de 1,5 salário mínimo até 2,0 salários mínimos.' " + "when Q006 = 'E' then 'Acima de 2,0 salários mínimos até 2,5 salários mínimos.' " + "when Q006 = 'F' then 'Acima de 2,5 salários mínimos até 3,0 salários mínimos.' " + "when Q006 = 'G' then 'Acima de 3,0 salários mínimos até 4,0 salários mínimos.' " + "when Q006 = 'H' then 'Acima de 4,0 salários mínimos até 5,0 salários mínimos.' " + "when Q006 = 'I' then 'Acima de 5,0 salários mínimos até 6,0 salários mínimos.' " + "when Q006 = 'J' then 'Acima de 6,0 salários mínimos até 7,0 salários mínimos.' " + "when Q006 = 'K' then 'Acima de 7,0 salários mínimos até 8,0 salários mínimos.' " + "when Q006 = 'L' then 'Acima de 8,0 salários mínimos até 9,0 salários mínimos.' " + "when Q006 = 'M' then 'Acima de 9,0 salários mínimos até 10,0 salários mínimos.' " + "when Q006 = 'N' then 'Acima de 10,0 salários mínimos até 12,0 salários mínimos.' " + "when Q006 = 'O' then 'Acima de 12,0 salários mínimos até 15,0 salários mínimos.' " + "when Q006 = 'P' then 'Acima de 15,0 salários mínimos até 20,0 salários mínimos.' " + "when Q006 = 'Q' then 'Acima de 20 salários mínimos.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna RENDA_MENSAL_FAMILIAR - 2020-08-20 17:56:08.970667.
Criação da coluna com sucesso - 2020-08-20 17:56:09.019536.


In [91]:
print(f'Criação da coluna RESIDENCIA_CELULAR - {datetime.now()}.')

df_enem_q022 = df_enem_q006.withColumn('RESIDENCIA_CELULAR', expr("case when Q022 = 'A' then 'Não.' " + "when Q022 = 'B' then 'Sim, um.' " + "when Q022 = 'C' then 'Sim, dois.' " + "when Q022 = 'D' then 'Sim, três.' " + "when Q022 = 'E' then 'Sim, quatro ou mais.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna RESIDENCIA_CELULAR - 2020-08-20 17:56:09.049456.
Criação da coluna com sucesso - 2020-08-20 17:56:09.107301.


In [92]:
print(f'Criação da coluna RESIDENCIA_COMPUTADOR - {datetime.now()}.')

df_enem_q024 = df_enem_q022.withColumn('RESIDENCIA_COMPUTADOR', expr("case when Q024 = 'A' then 'Não.' " + "when Q024 = 'B' then 'Sim, um.' " + "when Q024 = 'C' then 'Sim, dois.' " + "when Q024 = 'D' then 'Sim, três.' " + "when Q024 = 'E' then 'Sim, quatro ou mais.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna RESIDENCIA_COMPUTADOR - 2020-08-20 17:56:09.127249.
Criação da coluna com sucesso - 2020-08-20 17:56:09.205041.


In [93]:
print(f'Criação da coluna RESIDENCIA_INTERNET - {datetime.now()}.')

df_enem_q025 = df_enem_q024.withColumn('RESIDENCIA_INTERNET', expr("case when Q025 = 'A' then 'Não.' " + "when Q025 = 'B' then 'Sim.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

Criação da coluna RESIDENCIA_INTERNET - 2020-08-20 17:56:09.221995.
Criação da coluna com sucesso - 2020-08-20 17:56:09.267872.


In [94]:
# Criação do dataframe df_enem_analise para análise exploratória.

print(f'Criação do Dataframe para análise exploratória df_enem_analise - {datetime.now()}.')

df_enem_analise = df_enem_q025

print(f'Dataframe df_enem_analise criado com sucesso - {datetime.now()}.')

Criação do Dataframe para análise exploratória df_enem_analise - 2020-08-20 17:56:09.282832.
Dataframe df_enem_analise criado com sucesso - 2020-08-20 17:56:09.282832.


In [95]:
# Apresentação da estutura do dataframe df_enem_analise

print(f'Apresentação da estutura do dataframe df_enem_analise - {datetime.now()}.')

df_enem_analise.printSchema()

Apresentação da estutura do dataframe df_enem_analise - 2020-08-20 17:56:09.298789.
root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullabl

In [103]:
# Criar tabela temporaria e usar sql para consulta

print(f'Apresentação do processo de criação de uma tabela temporária para utilização do SQL - {datetime.now()}.')

df_enem_analise.createOrReplaceTempView('enemtb')

print(f'Criação da tabela temporaria com sucesso - {datetime.now()}.')

Apresentação do processo de criação de uma tabela temporária para utilização do SQL - 2020-08-20 18:11:16.344847.
Criação da tabela temporaria com sucesso - 2020-08-20 18:11:16.531062.


In [107]:
# Realização da Análise Exploratória

print('Quantidade de inscriçoes por Ano.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT NU_ANO AS ANO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY NU_ANO ORDER BY NU_ANO, COUNT(*)').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Ano.

Consulta inciada -  2020-08-20 18:20:43.943536
+----+--------------+
| ANO|QTDE_INSCRICAO|
+----+--------------+
|2015|       7746427|
|2016|       8627367|
|2017|       6731341|
|2018|       5513747|
|2019|       5095270|
+----+--------------+

Consulta realizada com sucesso -  2020-08-20 18:22:18.015673


In [108]:
print('Quantidade de inscriçoes por Sexo.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SEXO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SEXO ORDER BY COUNT(*)').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Sexo.

Consulta inciada -  2020-08-20 18:22:18.046586
+---------+--------------+
|     SEXO|QTDE_INSCRICAO|
+---------+--------------+
| Feminino|      19679368|
|Masculino|      14034784|
+---------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:23:55.740218


In [112]:
print('Quantidade de inscriçoes por Estado Civil.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT ESTADO_CIVIL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY ESTADO_CIVIL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Estado Civil.

Consulta inciada -  2020-08-20 18:34:29.682818
+--------------------+--------------+
|        ESTADO_CIVIL|QTDE_INSCRICAO|
+--------------------+--------------+
|            Solteiro|      24197550|
|Casado(a)/Mora co...|       7350094|
|                    |       1199287|
|Divorciado(a)/Des...|        829816|
|            Viúvo(a)|        137405|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:36:09.753529


In [114]:
print('Quantidade de inscriçoes por Raça.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT RACA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY RACA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Raça.

Consulta inciada -  2020-08-20 18:38:03.883448
+-------------+--------------+
|         RACA|QTDE_INSCRICAO|
+-------------+--------------+
|       Branca|      32435187|
|        Preta|       1135510|
|      Amarela|         85678|
|        Parda|         40436|
|Não declarado|         17341|
+-------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:39:48.101416


In [115]:
print('Quantidade de inscriçoes por Nacionalidade.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT NACIONALIDADE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY NACIONALIDADE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Nacionalidade.

Consulta inciada -  2020-08-20 18:39:48.241564
+--------------------+--------------+
|       NACIONALIDADE|QTDE_INSCRICAO|
+--------------------+--------------+
|       Brasileiro(a)|      32435187|
|Brasileiro(a) Nat...|       1135510|
|Brasileiro(a) Nat...|         85678|
|      Estrangeiro(a)|         40436|
|       Não informado|         17341|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:41:31.062981


In [116]:
print('Quantidade de inscriçoes por Situação Conclusão Ensino Médio.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SITUACAO_CONCLUSAO_ENSINO_MEDIO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SITUACAO_CONCLUSAO_ENSINO_MEDIO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Situação Conclusão Ensino Médio.

Consulta inciada -  2020-08-20 18:41:31.141503
+-------------------------------+--------------+
|SITUACAO_CONCLUSAO_ENSINO_MEDIO|QTDE_INSCRICAO|
+-------------------------------+--------------+
|           Já concluí o Ensi...|      19919399|
|           Estou cursando e ...|       8424851|
|           Estou cursando e ...|       4302929|
|           Não concluí e não...|       1066973|
+-------------------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:43:16.222189


In [117]:
print('Quantidade de inscriçoes por Tipo Escola.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT TIPO_ESCOLA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY TIPO_ESCOLA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Tipo Escola.

Consulta inciada -  2020-08-20 18:43:16.298983
+-------------+--------------+
|  TIPO_ESCOLA|QTDE_INSCRICAO|
+-------------+--------------+
|Não Respondeu|      25526429|
|      Pública|       6780145|
|      Privada|       1149757|
|     Exterior|        257821|
+-------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:44:57.642937


In [118]:
print('Quantidade de inscriçoes por Tipo Ensino.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT TIPO_ENSINO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY TIPO_ENSINO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Tipo Ensino.

Consulta inciada -  2020-08-20 18:44:57.693802
+--------------------+--------------+
|         TIPO_ENSINO|QTDE_INSCRICAO|
+--------------------+--------------+
|                    |      22036422|
|      Ensino Regular|      10700506|
|Educação de Joven...|        917342|
|Educação Especial...|         59882|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:46:30.947633


In [119]:
print('Quantidade de inscriçoes por Treineiro.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT TREINEIRO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY TREINEIRO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Treineiro.

Consulta inciada -  2020-08-20 18:46:31.002489
+---------+--------------+
|TREINEIRO|QTDE_INSCRICAO|
+---------+--------------+
|      Não|      30127568|
|      Sim|       3586584|
+---------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:48:06.597728


In [120]:
print('Quantidade de inscriçoes por Dependência Administrativa Escola.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DEPENDENCIA_ADMINISTRATIVA_ESCOLA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DEPENDENCIA_ADMINISTRATIVA_ESCOLA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Dependência Administrativa Escola.

Consulta inciada -  2020-08-20 18:48:06.632636
+---------------------------------+--------------+
|DEPENDENCIA_ADMINISTRATIVA_ESCOLA|QTDE_INSCRICAO|
+---------------------------------+--------------+
|                                 |      25801465|
|                         Estadual|       6280273|
|                          Privada|       1337142|
|                          Federal|        212646|
|                        Municipal|         82626|
+---------------------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:49:43.195782


In [121]:
print('Quantidade de inscriçoes por Localização Escola.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LOCALIZACAO_ESCOLA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LOCALIZACAO_ESCOLA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Localização Escola.

Consulta inciada -  2020-08-20 18:49:43.284544
+------------------+--------------+
|LOCALIZACAO_ESCOLA|QTDE_INSCRICAO|
+------------------+--------------+
|                  |      25801465|
|            Urbana|       7665166|
|             Rural|        247521|
+------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:51:19.163088


In [122]:
print('Quantidade de inscriçoes por Baixa Visão.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT BAIXA_VISAO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY BAIXA_VISAO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Baixa Visão.

Consulta inciada -  2020-08-20 18:51:19.214948
+-----------+--------------+
|BAIXA_VISAO|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33673697|
|        Sim|         40455|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 18:52:53.753921


In [123]:
print('Quantidade de inscriçoes por Cegueira.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT CEGUEIRA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY CEGUEIRA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Cegueira.

Consulta inciada -  2020-08-20 19:11:10.096829
+--------+--------------+
|CEGUEIRA|QTDE_INSCRICAO|
+--------+--------------+
|     Não|      33709598|
|     Sim|          4554|
+--------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:12:47.403104


In [124]:
print('Quantidade de inscriçoes por Surdez.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SURDEZ, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SURDEZ ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Surdez.

Consulta inciada -  2020-08-20 19:45:13.514148
+------+--------------+
|SURDEZ|QTDE_INSCRICAO|
+------+--------------+
|   Não|      33705038|
|   Sim|          9114|
+------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:47:05.207211


In [125]:
print('Quantidade de inscriçoes por Deficiência Auditiva.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DEFICIENCIA_AUDITIVA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DEFICIENCIA_AUDITIVA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Deficiência Auditiva.

Consulta inciada -  2020-08-20 19:47:05.355812
+--------------------+--------------+
|DEFICIENCIA_AUDITIVA|QTDE_INSCRICAO|
+--------------------+--------------+
|                 Não|      33691704|
|                 Sim|         22448|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:48:50.955585


In [126]:
print('Quantidade de inscriçoes por Deficiência Auditiva.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DEFICIENCIA_AUDITIVA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DEFICIENCIA_AUDITIVA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Deficiência Auditiva.

Consulta inciada -  2020-08-20 19:48:50.989495
+--------------------+--------------+
|DEFICIENCIA_AUDITIVA|QTDE_INSCRICAO|
+--------------------+--------------+
|                 Não|      33691704|
|                 Sim|         22448|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:50:35.703288


In [127]:
print('Quantidade de inscriçoes por Surdo_Cegueira.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SURDO_CEGUEIRA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SURDO_CEGUEIRA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Surdo_Cegueira.

Consulta inciada -  2020-08-20 19:50:35.733206
+--------------+--------------+
|SURDO_CEGUEIRA|QTDE_INSCRICAO|
+--------------+--------------+
|           Não|      33714026|
|           Sim|           126|
+--------------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:52:23.838919


In [128]:
print('Quantidade de inscriçoes por Deficiência Física.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DEFICIENCIA_FISICA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DEFICIENCIA_FISICA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Deficiência Física.

Consulta inciada -  2020-08-20 19:52:23.870450
+------------------+--------------+
|DEFICIENCIA_FISICA|QTDE_INSCRICAO|
+------------------+--------------+
|               Não|      33643894|
|               Sim|         70258|
+------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:54:15.068543


In [129]:
print('Quantidade de inscriçoes por Deficiência Mental.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DEFICIENCIA_MENTAL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DEFICIENCIA_MENTAL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Deficiência Mental.

Consulta inciada -  2020-08-20 19:54:15.096466
+------------------+--------------+
|DEFICIENCIA_MENTAL|QTDE_INSCRICAO|
+------------------+--------------+
|               Não|      33704239|
|               Sim|          9913|
+------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:56:06.209781


In [130]:
print('Quantidade de inscriçoes por Deficit de Atenção.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DEFICIT_ATENCAO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DEFICIT_ATENCAO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Deficit de Atenção.

Consulta inciada -  2020-08-20 19:56:06.230725
+---------------+--------------+
|DEFICIT_ATENCAO|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33681678|
|            Sim|         32474|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 19:58:07.155794


In [131]:
print('Quantidade de inscriçoes por Dislexia.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DISLEXIA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DISLEXIA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Dislexia.

Consulta inciada -  2020-08-20 19:58:07.214636
+--------+--------------+
|DISLEXIA|QTDE_INSCRICAO|
+--------+--------------+
|     Não|      33707202|
|     Sim|          6950|
+--------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:00:02.635197


In [132]:
print('Quantidade de inscriçoes por Discalculia.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT DISCALCULIA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY DISCALCULIA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Discalculia.

Consulta inciada -  2020-08-20 20:00:02.710996
+-----------+--------------+
|DISCALCULIA|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33712676|
|        Sim|          1476|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:01:58.108508


In [133]:
print('Quantidade de inscriçoes por Autismo.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT AUTISMO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY AUTISMO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Autismo.

Consulta inciada -  2020-08-20 20:01:58.151394
+-------+--------------+
|AUTISMO|QTDE_INSCRICAO|
+-------+--------------+
|    Não|      33710842|
|    Sim|          3310|
+-------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:04:02.725202


In [134]:
print('Quantidade de inscriçoes por Visão Monocular.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT VISAO_MONOCULAR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY VISAO_MONOCULAR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Visão Monocular.

Consulta inciada -  2020-08-20 20:04:02.759111
+---------------+--------------+
|VISAO_MONOCULAR|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33703225|
|            Sim|         10927|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:06:09.429462


In [135]:
print('Quantidade de inscriçoes por Outra Deficiência.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT OUTRA_DEF, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY OUTRA_DEF ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Outra Deficiência.

Consulta inciada -  2020-08-20 20:06:09.510459
+---------+--------------+
|OUTRA_DEF|QTDE_INSCRICAO|
+---------+--------------+
|      Não|      33695615|
|      Sim|         18537|
+---------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:08:14.268236


In [136]:
print('Quantidade de inscriçoes por Gestante.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT GESTANTE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY GESTANTE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Gestante.

Consulta inciada -  2020-08-20 20:08:14.428329
+--------+--------------+
|GESTANTE|QTDE_INSCRICAO|
+--------+--------------+
|     Não|      33693332|
|     Sim|         20820|
+--------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:10:21.965373


In [137]:
print('Quantidade de inscriçoes por Lactante.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LACTANTE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LACTANTE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Lactante.

Consulta inciada -  2020-08-20 20:10:22.018232
+--------+--------------+
|LACTANTE|QTDE_INSCRICAO|
+--------+--------------+
|     Não|      33671090|
|     Sim|         43062|
+--------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:12:30.199405


In [138]:
print('Quantidade de inscriçoes por Idoso.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT IDOSO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY IDOSO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Idoso.

Consulta inciada -  2020-08-20 20:12:30.400421
+-----+--------------+
|IDOSO|QTDE_INSCRICAO|
+-----+--------------+
|  Não|      33710312|
|  Sim|          3840|
+-----+--------------+

Consulta realizada com sucesso -  2020-08-20 20:14:38.736152


In [139]:
print('Quantidade de inscriçoes por Esuda em Classe Hospitalar.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT ESTUDA_CLASSE_HOSPITALAR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY ESTUDA_CLASSE_HOSPITALAR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Esuda em Classe Hospitalar.

Consulta inciada -  2020-08-20 20:14:38.970016
+------------------------+--------------+
|ESTUDA_CLASSE_HOSPITALAR|QTDE_INSCRICAO|
+------------------------+--------------+
|                     Não|      33713999|
|                     Sim|           153|
+------------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:16:45.643642


In [140]:
print('Quantidade de inscriçoes por Esuda em Sem Recurso.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SEM_RECURSO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SEM_RECURSO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Esuda em Sem Recurso.

Consulta inciada -  2020-08-20 20:16:45.795240
+-----------+--------------+
|SEM_RECURSO|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33598836|
|        Sim|        115316|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:18:51.041388


In [141]:
print('Quantidade de inscriçoes por Braille.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT BRAILLE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY BRAILLE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Braille.

Consulta inciada -  2020-08-20 20:18:51.141636
+-------+--------------+
|BRAILLE|QTDE_INSCRICAO|
+-------+--------------+
|    Não|      33712089|
|    Sim|          2063|
+-------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:21:16.199067


In [142]:
print('Quantidade de inscriçoes por Ampliada 24.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT AMPLIADA_24, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY AMPLIADA_24 ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Ampliada 24.

Consulta inciada -  2020-08-20 20:21:16.317749
+-----------+--------------+
|AMPLIADA_24|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33706677|
|        Sim|          7475|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:23:21.637527


In [143]:
print('Quantidade de inscriçoes por Ampliada 18.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT AMPLIADA_18, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY AMPLIADA_18 ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Ampliada 18.

Consulta inciada -  2020-08-20 20:23:21.725999
+-----------+--------------+
|AMPLIADA_18|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33687767|
|        Sim|         26385|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:25:24.822947


In [144]:
print('Quantidade de inscriçoes por Ledor.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LEDOR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LEDOR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Ledor.

Consulta inciada -  2020-08-20 20:25:24.878799
+-----+--------------+
|LEDOR|QTDE_INSCRICAO|
+-----+--------------+
|  Não|      33682753|
|  Sim|         31399|
+-----+--------------+

Consulta realizada com sucesso -  2020-08-20 20:27:27.830101


In [145]:
print('Quantidade de inscriçoes por Acesso.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT ACESSO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY ACESSO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Acesso.

Consulta inciada -  2020-08-20 20:27:27.984722
+------+--------------+
|ACESSO|QTDE_INSCRICAO|
+------+--------------+
|   Não|      33653419|
|   Sim|         60733|
+------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:29:39.484841


In [146]:
print('Quantidade de inscriçoes por Transcrição.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT TRANSCRICAO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY TRANSCRICAO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Transcrição.

Consulta inciada -  2020-08-20 20:29:39.508772
+-----------+--------------+
|TRANSCRICAO|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33683543|
|        Sim|         30609|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:31:52.232778


In [147]:
print('Quantidade de inscriçoes por Libras.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LIBRAS, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LIBRAS ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Libras.

Consulta inciada -  2020-08-20 20:31:52.281648
+------+--------------+
|LIBRAS|QTDE_INSCRICAO|
+------+--------------+
|   Não|      33704150|
|   Sim|         10002|
+------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:34:06.949538


In [148]:
print('Quantidade de inscriçoes por Leitura Labial.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LEITURA_LABIAL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LEITURA_LABIAL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Leitura Labial.

Consulta inciada -  2020-08-20 20:34:06.974458
+--------------+--------------+
|LEITURA_LABIAL|QTDE_INSCRICAO|
+--------------+--------------+
|           Não|      33708872|
|           Sim|          5280|
+--------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:36:10.007768


In [149]:
print('Quantidade de inscriçoes por Mesa Cadeira Rodas.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MESA_CADEIRA_RODAS, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MESA_CADEIRA_RODAS ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Mesa Cadeira Rodas.

Consulta inciada -  2020-08-20 20:36:10.054059
+------------------+--------------+
|MESA_CADEIRA_RODAS|QTDE_INSCRICAO|
+------------------+--------------+
|               Não|      33702892|
|               Sim|         11260|
+------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:38:15.166755


In [150]:
print('Quantidade de inscriçoes por Mesa Cadeira Separada.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MESA_CADEIRA_SEPARADA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MESA_CADEIRA_SEPARADA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Mesa Cadeira Separada.

Consulta inciada -  2020-08-20 20:38:15.201183
+---------------------+--------------+
|MESA_CADEIRA_SEPARADA|QTDE_INSCRICAO|
+---------------------+--------------+
|                  Não|      33697178|
|                  Sim|         16974|
+---------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:40:19.990536


In [151]:
print('Quantidade de inscriçoes por Apoio de Perna.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT APOIO_PERNA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY APOIO_PERNA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Apoio de Perna.

Consulta inciada -  2020-08-20 20:40:20.023444
+-----------+--------------+
|APOIO_PERNA|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33700885|
|        Sim|         13267|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:42:34.936812


In [152]:
print('Quantidade de inscriçoes por Guia Iterprete.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT GUIA_INTERPRETE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY GUIA_INTERPRETE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Guia Iterprete.

Consulta inciada -  2020-08-20 20:42:34.983702
+---------------+--------------+
|GUIA_INTERPRETE|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33714101|
|            Sim|            51|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:44:45.464011


In [153]:
print('Quantidade de inscriçoes por Pedido de Computador.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT COMPUTADOR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY COMPUTADOR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Pedido de Computador.

Consulta inciada -  2020-08-20 20:44:45.501910
+----------+--------------+
|COMPUTADOR|QTDE_INSCRICAO|
+----------+--------------+
|       Não|      33714118|
|       Sim|            34|
+----------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:46:57.898228


In [154]:
print('Quantidade de inscriçoes por Cadeira Especial.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT CADEIRA_ESPECIAL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY CADEIRA_ESPECIAL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Cadeira Especial.

Consulta inciada -  2020-08-20 20:46:57.939954
+----------------+--------------+
|CADEIRA_ESPECIAL|QTDE_INSCRICAO|
+----------------+--------------+
|             Não|      33714031|
|             Sim|           121|
+----------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:49:07.914224


In [155]:
print('Quantidade de inscriçoes por Cadeira Canhoto.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT CADEIRA_CANHOTO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY CADEIRA_CANHOTO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Cadeira Canhoto.

Consulta inciada -  2020-08-20 20:49:07.941678
+---------------+--------------+
|CADEIRA_CANHOTO|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33714030|
|            Sim|           122|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:51:15.954082


In [156]:
print('Quantidade de inscriçoes por Cadeira Canhoto.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT CADEIRA_CANHOTO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY CADEIRA_CANHOTO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Cadeira Canhoto.

Consulta inciada -  2020-08-20 20:51:15.987994
+---------------+--------------+
|CADEIRA_CANHOTO|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33714030|
|            Sim|           122|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:53:19.565908


In [157]:
print('Quantidade de inscriçoes por Cadeira Acolchoada.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT CADEIRA_ACOLCHOADA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY CADEIRA_ACOLCHOADA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Cadeira Acolchoada.

Consulta inciada -  2020-08-20 20:53:19.601812
+------------------+--------------+
|CADEIRA_ACOLCHOADA|QTDE_INSCRICAO|
+------------------+--------------+
|               Não|      33713388|
|               Sim|           764|
+------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:55:20.148980


In [158]:
print('Quantidade de inscriçoes por Prova Deitada.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PROVA_DEITADO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PROVA_DEITADO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Prova Deitada.

Consulta inciada -  2020-08-20 20:55:20.170403
+-------------+--------------+
|PROVA_DEITADO|QTDE_INSCRICAO|
+-------------+--------------+
|          Não|      33714131|
|          Sim|            21|
+-------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:57:22.286199


In [159]:
print('Quantidade de inscriçoes por Mobiliario Obeso.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MOBILIARIO_OBESO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MOBILIARIO_OBESO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Mobiliario Obeso.

Consulta inciada -  2020-08-20 20:57:22.316118
+----------------+--------------+
|MOBILIARIO_OBESO|QTDE_INSCRICAO|
+----------------+--------------+
|             Não|      33713885|
|             Sim|           267|
+----------------+--------------+

Consulta realizada com sucesso -  2020-08-20 20:59:07.485538


In [160]:
print('Quantidade de inscriçoes por Lamina Overlay.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LAMINA_OVERLAY, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LAMINA_OVERLAY ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Lamina Overlay.

Consulta inciada -  2020-08-20 20:59:07.508479
+--------------+--------------+
|LAMINA_OVERLAY|QTDE_INSCRICAO|
+--------------+--------------+
|           Não|      33714070|
|           Sim|            82|
+--------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:00:40.480370


In [161]:
print('Quantidade de inscriçoes por Protetor Auricular.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PROTETOR_AURICULAR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PROTETOR_AURICULAR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Protetor Auricular.

Consulta inciada -  2020-08-20 21:00:40.507295
+------------------+--------------+
|PROTETOR_AURICULAR|QTDE_INSCRICAO|
+------------------+--------------+
|               Não|      33714130|
|               Sim|            22|
+------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:02:14.508058


In [162]:
print('Quantidade de inscriçoes por Medidor Glicose.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MEDIDOR_GLICOSE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MEDIDOR_GLICOSE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Medidor Glicose.

Consulta inciada -  2020-08-20 21:02:14.528006
+---------------+--------------+
|MEDIDOR_GLICOSE|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33712326|
|            Sim|          1826|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:03:52.305357


In [163]:
print('Quantidade de inscriçoes por Máquina de Braile.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MAQUINA_BRAILE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MAQUINA_BRAILE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Máquina de Braile.

Consulta inciada -  2020-08-20 21:03:52.343290
+--------------+--------------+
|MAQUINA_BRAILE|QTDE_INSCRICAO|
+--------------+--------------+
|           Não|      33713607|
|           Sim|           545|
+--------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:05:25.565975


In [164]:
print('Quantidade de inscriçoes por Soroban.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SOROBAN, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SOROBAN ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Soroban.

Consulta inciada -  2020-08-20 21:05:25.586920
+-------+--------------+
|SOROBAN|QTDE_INSCRICAO|
+-------+--------------+
|    Não|      33713939|
|    Sim|           213|
+-------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:07:00.001411


In [165]:
print('Quantidade de inscriçoes por Marca Passo.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MARCA_PASSO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MARCA_PASSO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Marca Passo.

Consulta inciada -  2020-08-20 21:07:00.032326
+-----------+--------------+
|MARCA_PASSO|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33714102|
|        Sim|            50|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:08:34.208485


In [166]:
print('Quantidade de inscriçoes por Sonda.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SONDA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SONDA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Sonda.

Consulta inciada -  2020-08-20 21:08:34.228432
+-----+--------------+
|SONDA|QTDE_INSCRICAO|
+-----+--------------+
|  Não|      33714094|
|  Sim|            58|
+-----+--------------+

Consulta realizada com sucesso -  2020-08-20 21:10:08.446343


In [167]:
print('Quantidade de inscriçoes por Medicamentos.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MEDICAMENTOS, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MEDICAMENTOS ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Medicamentos.

Consulta inciada -  2020-08-20 21:10:08.469281
+------------+--------------+
|MEDICAMENTOS|QTDE_INSCRICAO|
+------------+--------------+
|         Não|      33713057|
|         Sim|          1095|
+------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:11:46.331199


In [168]:
print('Quantidade de inscriçoes por Sala Individual.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SALA_INDIVIDUAL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SALA_INDIVIDUAL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Sala Individual.

Consulta inciada -  2020-08-20 21:11:46.357689
+---------------+--------------+
|SALA_INDIVIDUAL|QTDE_INSCRICAO|
+---------------+--------------+
|            Não|      33712675|
|            Sim|          1477|
+---------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:13:18.294057


In [169]:
print('Quantidade de inscriçoes por Sala Especial.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SALA_ESPECIAL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SALA_ESPECIAL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Sala Especial.

Consulta inciada -  2020-08-20 21:13:18.308020
+-------------+--------------+
|SALA_ESPECIAL|QTDE_INSCRICAO|
+-------------+--------------+
|          Não|      33712905|
|          Sim|          1247|
+-------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:14:51.789375


In [170]:
print('Quantidade de inscriçoes por Sala para Acompanhante.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT SALA_ACOMPANHANTE, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY SALA_ACOMPANHANTE ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Sala para Acompanhante.

Consulta inciada -  2020-08-20 21:14:51.816300
+-----------------+--------------+
|SALA_ACOMPANHANTE|QTDE_INSCRICAO|
+-----------------+--------------+
|              Não|      33713752|
|              Sim|           400|
+-----------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:16:25.120271


In [171]:
print('Quantidade de inscriçoes por Mobiliario Específico.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MOBILIARIO_ESPECIFICO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MOBILIARIO_ESPECIFICO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Mobiliario Específico.

Consulta inciada -  2020-08-20 21:16:25.138222
+---------------------+--------------+
|MOBILIARIO_ESPECIFICO|QTDE_INSCRICAO|
+---------------------+--------------+
|                  Não|      33714017|
|                  Sim|           135|
+---------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:17:59.390371


In [172]:
print('Quantidade de inscriçoes por Nome Social.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT NOME_SOCIAL, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY NOME_SOCIAL ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Nome Social.

Consulta inciada -  2020-08-20 21:17:59.419295
+-----------+--------------+
|NOME_SOCIAL|QTDE_INSCRICAO|
+-----------+--------------+
|        Não|      33712493|
|        Sim|          1659|
+-----------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:19:37.218506


In [173]:
print('Quantidade de inscriçoes por Presença Prova Ciências Natureza.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PRESENCA_CIENCIAS_NATUREZA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PRESENCA_CIENCIAS_NATUREZA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Presença Prova Ciências Natureza.

Consulta inciada -  2020-08-20 21:19:37.254605
+--------------------------+--------------+
|PRESENCA_CIENCIAS_NATUREZA|QTDE_INSCRICAO|
+--------------------------+--------------+
|         Presente na prova|      23807309|
|            Faltou à prova|       9894904|
|        Eliminado na prova|         10840|
|                          |          1099|
+--------------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:21:12.530540


In [174]:
print('Quantidade de inscriçoes por Presença Prova Ciências Humanas.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PRESENCA_CIENCIAS_HUMANAS, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PRESENCA_CIENCIAS_HUMANAS ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Presença Prova Ciências Humanas.

Consulta inciada -  2020-08-20 21:21:12.549488
+-------------------------+--------------+
|PRESENCA_CIENCIAS_HUMANAS|QTDE_INSCRICAO|
+-------------------------+--------------+
|        Presente na prova|      24527054|
|           Faltou à prova|       9170610|
|       Eliminado na prova|         15389|
|                         |          1099|
+-------------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:22:47.588751


In [175]:
print('Quantidade de inscriçoes por Presença Prova Linguagens e Códigos.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PRESENCA_LINGUAGENS_CODIGOS, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PRESENCA_LINGUAGENS_CODIGOS ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Presença Prova Linguagens e Códigos.

Consulta inciada -  2020-08-20 21:22:47.609694
+---------------------------+--------------+
|PRESENCA_LINGUAGENS_CODIGOS|QTDE_INSCRICAO|
+---------------------------+--------------+
|          Presente na prova|      24243348|
|             Faltou à prova|       9452565|
|         Eliminado na prova|         17164|
|                           |          1075|
+---------------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:24:25.244317


In [176]:
print('Quantidade de inscriçoes por Presença Prova Matemática.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PRESENCA_MATEMATICA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PRESENCA_MATEMATICA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Presença Prova Matemática.

Consulta inciada -  2020-08-20 21:24:25.263266
+-------------------+--------------+
|PRESENCA_MATEMATICA|QTDE_INSCRICAO|
+-------------------+--------------+
|  Presente na prova|      23523603|
|     Faltou à prova|      10176859|
| Eliminado na prova|         12615|
|                   |          1075|
+-------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:26:02.108106


In [177]:
print('Quantidade de inscriçoes por Escolha Línguas.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT LINGUA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY LINGUA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Escolha Línguas.

Consulta inciada -  2020-08-20 21:26:02.138027
+--------+--------------+
|  LINGUA|QTDE_INSCRICAO|
+--------+--------------+
|Espanhol|      18995697|
|  Inglês|      14718455|
+--------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:27:36.830387


In [178]:
print('Quantidade de inscriçoes por Status Redação.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT STATUS_REDACAO, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY STATUS_REDACAO ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Status Redação.

Consulta inciada -  2020-08-20 21:27:36.846338
+--------------------+--------------+
|      STATUS_REDACAO|QTDE_INSCRICAO|
+--------------------+--------------+
|       Sem problemas|      23474861|
|                    |       7366434|
|           Em Branco|       2333045|
|        Fuga ao tema|        373136|
|Cópia Texto Motiv...|         55760|
|  Texto insuficiente|         41654|
|  Parte desconectada|         32730|
|Não atendimento a...|         20782|
|             Anulada|         15750|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:29:12.020706


In [179]:
print('Quantidade de inscriçoes por Pai Estudou?.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT PAI_ESTUDOU, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY PAI_ESTUDOU ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Pai Estudou?.

Consulta inciada -  2020-08-20 21:29:12.047633
+--------------------+--------------+
|         PAI_ESTUDOU|QTDE_INSCRICAO|
+--------------------+--------------+
|Completou o Ensin...|       8116484|
|Não completou a 4...|       7873004|
|Completou a 4ª sé...|       5084983|
|Completou a 8ª sé...|       4074944|
|            Não sei.|       3170233|
|Completou a Facul...|       2114497|
|      Nunca estudou.|       2076754|
|Completou a Pós-g...|       1202565|
|                    |           688|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:30:48.782228


In [180]:
print('Quantidade de inscriçoes por Mãe Estudou?.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT MAE_ESTUDOU, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY MAE_ESTUDOU ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Mãe Estudou?.

Consulta inciada -  2020-08-20 21:30:48.804167
+--------------------+--------------+
|         MAE_ESTUDOU|QTDE_INSCRICAO|
+--------------------+--------------+
|Completou o Ensin...|      10062314|
|Não completou a 4...|       6238429|
|Completou a 4ª sé...|       4993957|
|Completou a 8ª sé...|       4668985|
|Completou a Facul...|       2854443|
|Completou a Pós-g...|       2107605|
|      Nunca estudou.|       1533813|
|            Não sei.|       1253916|
|                    |           690|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:32:24.940905


In [181]:
print('Quantidade de inscriçoes por Quantidade de Pessoas na Residência.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT QTDE_MORADOR_RESIDENCIA, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY QTDE_MORADOR_RESIDENCIA ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Quantidade de Pessoas na Residência.

Consulta inciada -  2020-08-20 21:32:24.958857
+-----------------------+--------------+
|QTDE_MORADOR_RESIDENCIA|QTDE_INSCRICAO|
+-----------------------+--------------+
|                      4|      10520748|
|                      3|       8272709|
|                      5|       5752725|
|                      2|       4125748|
|                      6|       2275631|
|                      7|        968899|
|   1, pois moro sozi...|        949920|
|                      8|        443489|
|                      9|        178596|
|                     10|        117303|
|                     11|         39419|
|                     12|         27269|
|                     13|         12265|
|                     14|          6865|
|                     20|          6265|
|                     15|          6103|
|                     16|          2996|
|                     18|          2261|
|                     17|

In [182]:
print('Quantidade de inscriçoes por Renda Mensal Familiar.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT RENDA_MENSAL_FAMILIAR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY RENDA_MENSAL_FAMILIAR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Renda Mensal Familiar.

Consulta inciada -  2020-08-20 21:33:58.071181
+---------------------+--------------+
|RENDA_MENSAL_FAMILIAR|QTDE_INSCRICAO|
+---------------------+--------------+
| Acima de 1,0 salá...|       8676283|
| Até 1,0 salário m...|       8562846|
| Acima de 1,5 salá...|       3886465|
| Acima de 2,0 salá...|       2926930|
| Acima de 3,0 salá...|       1968376|
| Acima de 2,5 salá...|       1846761|
|       Nenhuma Renda.|       1241992|
| Acima de 4,0 salá...|       1199179|
| Acima de 5,0 salá...|        866593|
| Acima de 6,0 salá...|        564727|
| Acima de 7,0 salá...|        361117|
| Acima de 10,0 sal...|        333550|
| Acima de 20 salár...|        315608|
| Acima de 12,0 sal...|        266142|
| Acima de 8,0 salá...|        247428|
| Acima de 15,0 sal...|        230354|
| Acima de 9,0 salá...|        219101|
|                     |           700|
+---------------------+--------------+

Consulta realizada com sucesso -  2020-08

In [183]:
print('Quantidade de inscriçoes por Residência com Celular.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT RESIDENCIA_CELULAR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY RESIDENCIA_CELULAR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Residência com Celular.

Consulta inciada -  2020-08-20 21:35:34.818243
+--------------------+--------------+
|  RESIDENCIA_CELULAR|QTDE_INSCRICAO|
+--------------------+--------------+
|          Sim, dois.|      10926685|
|          Sim, três.|       8486907|
|            Sim, um.|       7810792|
|Sim, quatro ou mais.|       5759039|
|                Não.|        729975|
|                    |           754|
+--------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:37:08.699248


In [184]:
print('Quantidade de inscriçoes por Residência com Computador.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT RESIDENCIA_COMPUTADOR, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY RESIDENCIA_COMPUTADOR ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Residência com Computador.

Consulta inciada -  2020-08-20 21:37:08.720192
+---------------------+--------------+
|RESIDENCIA_COMPUTADOR|QTDE_INSCRICAO|
+---------------------+--------------+
|             Sim, um.|      16698903|
|                 Não.|      13316634|
|           Sim, dois.|       2617783|
|           Sim, três.|        755001|
| Sim, quatro ou mais.|        325069|
|                     |           762|
+---------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:38:42.488968


In [185]:
print('Quantidade de inscriçoes por Residência com Internet.')
print('')
print(f'Consulta inciada -  {datetime.now()}')

sqlContext.sql('SELECT RESIDENCIA_INTERNET, COUNT(*) AS QTDE_INSCRICAO FROM enemtb GROUP BY RESIDENCIA_INTERNET ORDER BY COUNT(*) DESC').show()

print(f'Consulta realizada com sucesso -  {datetime.now()}')

Quantidade de inscriçoes por Residência com Internet.

Consulta inciada -  2020-08-20 21:38:42.509203
+-------------------+--------------+
|RESIDENCIA_INTERNET|QTDE_INSCRICAO|
+-------------------+--------------+
|               Sim.|      23073704|
|               Não.|      10639681|
|                   |           767|
+-------------------+--------------+

Consulta realizada com sucesso -  2020-08-20 21:40:18.781537
